<a href="https://colab.research.google.com/github/james00000007/Google-Colaboratory/blob/main/Amazing_Colab_Notebook%E2%9C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown  ← Current Python Version
!python -V

In [ ]:
#@markdown <h3> ⬅ Prevent Google Colab from disconnecting | AUTO RECONNECT
import IPython
from google.colab import output
 
display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
  btn = document.getElementById('ok')
  if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
    
setInterval(ClickConnect,60000)
'''))

In [ ]:
#@markdown <h3>⬅ Mount Google Drive</h3>
import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output
 
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPath()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))
 
while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

In [ ]:
#@markdown <h3>⬅ To Mount Another Gdrive!</h3>
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

## <img src='https://colab.research.google.com/img/colab_favicon_256px.png' height="30" alt="colab" />  __Torrent to Gdrive__



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

from IPython.display import HTML, clear_output

clear_output()

print("Server Started Successfully")



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Upload torrent file</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>



from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/Shareddrives/Texas Woman's University Denton",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>


params = {"save_path": "/content/drive/Shareddrives}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


## <img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="30" alt="RcloneLab"/>

1️⃣ Click on the Cell below to upload your configuration file <br> <b> rclone.conf </b> if you already have one, or go to cell 3 to create your config.

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅ Execute Upload Local File
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "mnc"  # @param {type:"string"}
QUERY_PATTERN = ""  # @param {type:"string"}
# @markdown #### <font size=1px>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #
 
from os import path as _p
 
if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec
 
    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec
 
import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession
 
 
def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )
 
 
def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass
 
    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("\nUpload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")
 
 
if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run
 
  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))
 
from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)
 
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
 
prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
 
if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'
 
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'
 
    if runSh(cmd, shell=True) == 0:
      print(f"🤩 Mount success!👍 - \t{mPoint}")
    else:
      print(f"🤔 Mount failed! - \t{mPoint}")
 
def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)
 
ShowAC()

5️⃣<br><img src='https://1.bp.blogspot.com/-q2TmGe-HsDo/XWZgsRtILbI/AAAAAAAADOM/PnoCmKIBujcfZm94pN9nSgURf7sdk1rYACLcBGAs/s640/rclone-new-logo.png' height="30" alt="RcloneCoLab"/><br>You can use a Free Ngrok Token, but to make a copy for several hours, you will need to make a free account at  https://dashboard.ngrok.com/get-started/setup
<font color=yellow size=5><center><b>**Rclone Web UI**</center></b></font>

In [ ]:
#@markdown <center><h3>Rclone webui</h3></center><br>
import os, signal
import random
import string
import urllib.request
from IPython.display import HTML, clear_output
from urllib.parse import urlparse
import time

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : user<br>&emsp;password : pass
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
    checkAvailable
)
from rlab_utils import (
    displayOutput,
    checkAvailable,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)

loadingAn()
prepareSession()
pid = findProcess("rclone", "rcd", isPid=True)
try:
  os.kill(int(pid), signal.SIGTERM)
except TypeError:
  pass
cmd = "rclone rcd --rc-web-gui --rc-addr :5572" \
      " --rc-serve" \
      " --rc-user=user --rc-pass=pass" \
      " --rc-no-auth" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --transfers 16' \
      " &"
runSh(cmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rclone', 5572, 'http'],
                            ['filebrowser', 4000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/filebrowserRclone.yml", 4099]
).start('rclone', displayB=False)
# output
clear_output()
displayUrl(Server, pNamU='Rclone webUI : ', 
           ExUrl=fr"https://user:pass@{urlparse(Server['url']).hostname}")

In [ ]:
#@markdown <h3>⬅️ Run This Cell to Execute RcloneLab</h3>
# ============================= FORM ============================= #
Mode = "Copy" #@param ["Config", "Copy", "Move", "Mount", "Sync", "Checker", "Deduplicate", "Remove Empty Directories", "Empty Trash", "qBittorrent"]
Compare = "Size & Mod-Time" #@param ["Size & Mod-Time", "Size & Checksum", "Only Mod-Time", "Only Size", "Only Checksum"]
Source = "" #@param {type:"string"}
Destination = "" #@param {type:"string"}
Transfers = 20 #@param {type:"slider", min:1, max:20, step:1}
Checkers = 20 #@param {type:"slider", min:1, max:20, step:1}
#@markdown ---

#@markdown <center><h3><font color="#DFFF00"><b>⚙️ Global Configuration ⚙️</b></font></h3></center>
Simple_Ouput = True #@param {type:"boolean"}
Skip_files_that_are_newer_on_the_destination = False #@param {type:"boolean"}
Skip_all_files_that_exist = False #@param {type:"boolean"}
Do_not_cross_filesystem_boundaries = False
Do_not_update_modtime_if_files_are_identical = False #@param {type:"boolean"}
Large_amount_of_files_optimization = False
Google_Drive_optimization = False #@param {type:"boolean"}
Google_Drive_server_side_copy = True #@param {type:"boolean"}
Dry_Run = False #@param {type:"boolean"}
Output_Log_File = "OFF" #@param ["OFF", "NOTICE", "INFO", "ERROR", "DEBUG"]
Extra_Arguments = "" #@param {type:"string"}

#@markdown <center><h3><font color="#00FFFF"><b>↪️ Sync Configuration ↩️</b></font></h3></center>
Sync_Mode = "Delete during transfer" #@param ["Delete during transfer", "Delete before transfering", "Delete after transfering"]
Track_Renames = False #@param {type:"boolean"}

#@markdown <center><h3><font color="#FF00FF"><b>💞 Deduplicate Configuration 💞</b></font></h3></center>
Deduplicate_Mode = "Interactive" #@param ["Interactive", "Skip", "First", "Newest", "Oldest", "Largest", "Rename"]
Deduplicate_Use_Trash = True #@param {type:"boolean"}
# ================================================================ #

import os, psutil; from IPython.display import HTML, clear_output; from IPython.utils.io import ask_yes_no

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

### Variable Declaration

# Optimized for Google Colaboratory
if Google_Drive_optimization:
  bufferC = "--buffer-size 128M"
else:
  bufferC = "--buffer-size 192M"

if Compare == "Size & Checksum":
  compareC = "-c"
elif Compare == "Only Mod-Time":
  compareC = "--ignore-size"
elif Compare == "Only Size":
  compareC = "--size-only"
elif Compare == "Only Checksum":
  compareC = "-c --ignore-size"
else:
  compareC = ""

transfersC = "--transfers "+str(Transfers)
checkersC = "--checkers "+str(Checkers)

if Skip_files_that_are_newer_on_the_destination:
  skipnewC = "-u"
else:
  skipnewC = ""
  
if Skip_all_files_that_exist:
  skipexistC = "--ignore-existing"
else:
  skipexistC = ""
  
if Do_not_cross_filesystem_boundaries:
  nocrossfilesystemC = "--one-file-system"
else:
  nocrossfilesystemC = ""
  
if Do_not_update_modtime_if_files_are_identical:
  noupdatemodtimeC = "--no-update-modtime"
else:
  noupdatemodtimeC = ""

if Large_amount_of_files_optimization:
  filesoptimizeC = "--fast-list"
else:
  filesoptimizeC = ""
  
if Google_Drive_optimization:
  driveoptimizeC = "--drive-chunk-size 128M --drive-acknowledge-abuse --drive-keep-revision-forever"
else:
  driveoptimizeC = ""

if Google_Drive_server_side_copy:
  drivecopyC = "--drive-server-side-across-configs=true"
elif Google_Drive_optimization:
  drivecopyC = "--disable copy"

if Dry_Run:
  dryrunC = "-n"
else:
  dryrunC = ""
  
if Output_Log_File != "OFF":
  statsC = "--log-file=/root/.rclone_log/rclone_log.txt"
else:
  if Simple_Ouput:
    statsC = "-v --stats-one-line --stats=5s"
  else:
    statsC = "-v --stats=5s"
  
if Output_Log_File == "INFO":
  loglevelC = "--log-level INFO"
elif Output_Log_File == "ERROR":
  loglevelC = "--log-level ERROR"
elif Output_Log_File == "DEBUG":
  loglevelC = "--log-level DEBUG"
else:
  loglevelC = ""

if Sync_Mode == "Delete during transfer":
  syncmodeC = "--delete-during"
elif Sync_Mode == "Delete before transfering":
  syncmodeC = "--delete-before"
elif Sync_Mode == "Delete after transfering":
  syncmodeC = "--delete-after"
  
if Track_Renames:
  trackrenamesC = "--track-renames"
else:
  trackrenamesC = ""
  
if Deduplicate_Mode == "Interactive":
  deduplicateC = "interactive"
elif Deduplicate_Mode == "Skip":
  deduplicateC = "skip"
elif Deduplicate_Mode == "First":
  deduplicateC = "first"
elif Deduplicate_Mode == "Newest":
  deduplicateC = "newest"
elif Deduplicate_Mode == "Oldest":
  deduplicateC = "oldest"
elif Deduplicate_Mode == "Largest":
  deduplicateC = "largest"
elif Deduplicate_Mode == "Rename":
  deduplicateC = "rename"
  
if Deduplicate_Use_Trash:
  deduplicatetrashC = ""
else:
  deduplicatetrashC = "--drive-use-trash=false"
  
### rclone Execution

if Output_Log_File != "OFF" and Mode != "Config":
  get_ipython().system_raw("mkdir -p -m 666 /root/.rclone_log/")
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Logging enables, rclone will not output log through the terminal, please wait until finished.</h2><br></center>"))

if Mode == "Config":
  !rclone --config=/root/.rclone.conf config
elif Mode == "Copy":
  !rclone --config=/root/.rclone.conf copy "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Move":
  !rclone --config=/root/.rclone.conf move "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Mount":
  try:
    if not CheckProcess("rclone", "/content/" + Source.split(":")[0] + "/"):
      if not os.path.exists("/content/" + Source.split(":")[0] + "/"):
        get_ipython().system_raw("mkdir -p -m 666 '/content/" + Source.split(":")[0] + "/'")
      get_ipython().system_raw("rclone --config=/root/.rclone.conf mount '$Source' '/content/" + Source.split(":")[0] + "/' $loglevelC $noupdatemodtimeC $bufferC $filesoptimizeC --drive-chunk-size 128M --drive-acknowledge-abuse --drive-keep-revision-forever $Extra_Arguments &")
  except:
    pass
elif Mode == "Sync":
  !rclone --config=/root/.rclone.conf sync "$Source" "$Destination" $transfersC $checkersC $statsC $loglevelC $syncmodeC $trackrenamesC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Checker":
  !rclone --config=/root/.rclone.conf check "$Source" "$Destination" $checkersC $statsC $loglevelC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Deduplicate":
  !rclone --config=/root/.rclone.conf dedupe "$Source" $checkersC $statsC $loglevelC --dedupe-mode $deduplicateC $deduplicatetrashC $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments
elif Mode == "Remove Empty Directories":
  !rclone --config=/root/.rclone.conf rmdirs "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments
elif Mode == "Empty Trash":
  !rclone --config=/root/.rclone.conf cleanup "$Source" $statsC $loglevelC $dryrunC $Extra_Arguments
elif Mode == "qBittorrent":
  if Source == "":
    !chmod -R 666 /content/qBittorrent/ && rclone --config=/root/.rclone.conf move "/content/qBittorrent/" "$Destination" $transfersC $checkersC $statsC $loglevelC --delete-empty-src-dirs --exclude **/.unwanted/ $compareC $skipnewC $skipexistC $nocrossfilesystemC $noupdatemodtimeC $bufferC $filesoptimizeC $driveoptimizeC $drivecopyC $dryrunC $Extra_Arguments

### Log Output
  
if Output_Log_File != "OFF" and Mode != "Config" and os.path.isfile("/root/.rclone_log/rclone_log.txt"):
  ### Rename log file and output settings.
  get_ipython().system_raw("mv /root/.rclone_log/rclone_log.txt /root/.rclone_log/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).txt")
  with open("/root/.rclone_log/" + Mode + "_settings.txt", "w") as f:
    f.write("Mode: " + Mode + \
            "\nCompare: " + Compare + \
            "\nSource: \"" + Source + \
            "\"\nDestination: \"" + Destination + \
            "\"\nTransfers: " + str(Transfers) + \
            "\nCheckers: " + str(Checkers) + \
            "\nSkip files that are newer on the destination: " + str(Skip_files_that_are_newer_on_the_destination) + \
            "\nSkip all files that exist: " + str(Skip_all_files_that_exist) + \
            "\nDo not cross filesystem boundaries: " + str(Do_not_cross_filesystem_boundaries) + \
            "\nDo not update modtime if files are identical: " + str(Do_not_update_modtime_if_files_are_identical) + \
            "\nDry-Run: " + str(Dry_Run) + \
            "\nOutput Log Level: " + Output_Log_File + \
            "\nExtra Arguments: \"" + Extra_Arguments + \
            "\"\nSync Moden: " + Sync_Mode + \
            "\nTrack Renames: " + str(Track_Renames) + \
            "\nDeduplicate Mode: " + Deduplicate_Mode + \
            "\nDeduplicate Use Trash: " + str(Deduplicate_Use_Trash))
  ### Compressing log file.
  get_ipython().system_raw("rm -f /root/rclone_log.zip && zip -r -q -j -9 /root/rclone_log.zip /root/.rclone_log/ && rm -rf /root/.rclone_log/ && mkdir -p -m 666 /root/.rclone_log/")
  ### Send Log
  if os.path.isfile("/root/rclone_log.zip"):
    try:
      files.download("/root/rclone_log.zip")
      get_ipython().system_raw("rm -f /root/rclone_log.zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Sending log to your browser...</h2><br></center>"))
    except:
      get_ipython().system_raw("mv /root/rclone_log.zip /content/rclone_log_$(date +%Y-%m-%d_%H.%M.%S).zip")
      display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can use file explorer to download the log file.</h2><br><img src=\"https://raw.githubusercontent.com/Sparoney/RcloneLab/master/res/rclonelab/01.png\"><br></center>"))
  else:
    clear_output(wait=True)
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">There is no log file.</h2><br></center>"))

### Operation has been successfully completed.
if os.path.isfile("/usr/bin/rclone") == False:
  clear_output(wait=True)
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please run the installation cell above first.</h2><br></center>"))
elif Mode == "qBittorrent" and Source.strip() != "":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ Please empty the Source field in qBittorrent mode.</h2><br></center>"))
elif Mode == "Mount" and Source.strip() == "":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The Source field is empty.</h2><br></center>"))
elif Mode != "Config":
  display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">✅ Operation has been successfully completed.</h2><br></center>"))

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Download config file.
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG']
# ================================================================ #
from google.colab import files

def downloadFile():
  if MODE == "UTILS":
      filePath = "/root/.ipython/rlab_utils.py"
  elif MODE == "RCONFIG":
      filePath = f"{PATH_RClone_Config}/rclone.conf"
  else:
      pass
  try:
    files.download(filePath)
  except FileNotFoundError:
    print("File not found!")

if __name__ == "__main__":
  downloadFile()


##![alt text](https://mega.nz/favicon.ico?v=3) __TRANSFERE to MEGA__





In [ ]:
# @markdown <center><h2>INPUT YOUR MEGA ID</h2></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")


In [ ]:
import time
import subprocess
import contextlib
from IPython.display import clear_output
# @markdown <br><center><img src='https://www.impactcomputing.co.uk/uploads/files/services/managed-office-365.svg' height="100" alt="netdata"/></center>
# @markdown <center><h2>TRANSFERE NOW</h2></center><br>
# TRANSFERE NOW

PATH_TO_FILE = ""  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")


In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()



## <font color="#FF00FF"><b>💞 Retrieve Access Tokens for Google APIS  💞

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="30" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Retrieve access token for Google Drive API</h3></center><br>
client_id="" #@param {type:"string"}
client_secret="" #@param {type:"string"}

import subprocess
from google.colab import output

def formatx(i, key, j, k):
    val=response[i].split(": ")[1][1-j:-2+j+k]
    return key+": "+val

print("Go to this URL in a browser:: https://accounts.google.com/o/oauth2/auth?client_id={}&redirect_uri=urn:ietf:wg:oauth:2.0:oob&scope=https://www.googleapis.com/auth/drive&response_type=code".format(client_id))
auth_code=input("Enter your authorization code: ")
output.clear()
response = subprocess.check_output("curl -d client_id={} -d client_secret={} -d grant_type=authorization_code -d redirect_uri=urn:ietf:wg:oauth:2.0:oob -d code={} https://accounts.google.com/o/oauth2/token".format(client_id, client_secret, auth_code), shell=True).decode().split("\n")[1:6]
access_token, expires_in, refresh_token, scope, token_type = formatx(0, "access_token", 0, 0), formatx(1, "expires_in", 1, 0), formatx(2, "refresh_token", 0, 0), formatx(3, "scope", 0, 0), formatx(4, "token_type", 0, 1)

print(access_token); print("··········")
print(expires_in); print("··········")
print(refresh_token); print("··········")
print(scope); print("··········")
print(token_type); print("··········")
print("in_json_form:", "{"+", ".join("".join("".join(response).split("  ")).split(","))+"}")

## <font color="#00FFFF"><b>🔥 Transfer Between Drives  🔥

In [ ]:
#@markdown <center><h3><font color="#DFFF00"><b>⚙️ Input folder will be copied into the destination folder ⚙️
input_folder = "" #@param {type:"string"}
destination_folder = "" #@param {type:"string"}
!cp -a -v "$input_folder" "$destination_folder"
clear_output(wait=True)
print("Copied successfully!")

In [ ]:
#@markdown <center><h3><font color="#CC0000"><b>⚙️ Input file will be copied into the destination folder ⚙️
input_file = "" #@param {type:"string"}
destination_folder = "" #@param {type:"string"}
!cp -v "$input_file" "$destination_folder"
print("Copied successfully!")

## <img src='https://i.ibb.co/pKCdZhK/vpsmate-aria2-logo-1536x864.jpg' height="50" alt="aria2" />

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.ibb.co/pKCdZhK/vpsmate-aria2-logo-1536x864.jpg' height="50" alt="aria2"/></center>
#@markdown <center><h3>aria2<br />The next generation download utility.</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Ariang_WEBUI = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
# OUTPUT_DIR = ""  # @param {type:"string"}
PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/totalleecher/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")


from ttmg import (
    updateCheck,
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess
)

#if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
#    !kill -9 -1 &
#clear_output()
loadingAn()

# Ariang SETUP
runSh('sudo apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  BASE_URL = r"https://github.com/mayswind/AriaNg"
  LATEST_TAG = latestTag(BASE_URL)
  urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
              f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(urlF, 'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = urllib.request.urlopen(
          "https://trackerslist.com/best_aria2.txt"
      ).read().decode()
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"-j 20 " \
            r"-c " \
            fr"--bt-tracker={trackers} " \
            r"--bt-request-peer-speed-limit=0 " \
            r"--bt-max-peers=0 " \
            r"--seed-ratio=0.0 " \
            r"--max-connection-per-server=10 " \
            r"--min-split-size=10M " \
            r"--follow-torrent=mem " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http'], 
    ['Ariang', 8221, 'http']], 'jp', 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
if Aria2_rpc:
  data = Server.start('Aria2_rpc', displayB=False)
  Host = data['url'][7:] 
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
 
  data2 = Server.start('Ariang', displayB=False)
  displayUrl(data2, pNamU='Ariang : ')

if Aria2_rpc:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2qoOw0a">NB. How to setup this's config. [Click ME]</a></button><center>"""))


## <img src='https://i.ibb.co/pKCdZhK/vpsmate-aria2-logo-1536x864.jpg' height="30" alt="aira2"/> <font size=1px >___Direct link downloader / Remote Uploading___</font>

___Auto share link supported : Disk.yandex.com, Gdrive and youtube-dl all extractor...___

In [ ]:
%cd /content/drive/My Drive
%mkdir DirectDownload

#

URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path <b>/content/drive/My Drive/DirectDownload</b></h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ttmg.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py \
                    -O /root/.ipython/ttmg.py"
    run(split(shellCmd))
from ttmg import updateCheck, runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/drive/My Drive/DirectDownload"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("DirectDownload")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

##<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Pornhub-logo.svg/512px-Pornhub-logo.svg.png' height="30" alt="filebrowser" />  <h5>

In [ ]:
#@markdown <h3>⬅️ Click Here to Install Require Modules</h3>

#@title <font size="5">← ឵឵<i>Upgrade FFmpeg to v4.2.2</font> { vertical-output: true }
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip3 install youtube-dl')
os.system('pip3 install prettytable')
os.system('pip3 install bs4')
os.system('pip3 install requests')
%cd /content
os.system('git clone https://github.com/mariosemes/PornHub-downloader-python.git')

clear_output()
print("Install Finished")

In [ ]:
#@markdown <h3>⬅️ Click Here to download video</h3>

#@markdown - Single Download link Eg: https://www.pornhub.com/view_video.php?viewkey=ph5d69a2093729e
#@markdown - The batch option will ask you for the full path of your .txt file where you can import multiple URLs at once.Take care that every single URL in the .txt file is in his own row.



pornhub_url = '' #@param {type: "string"}
option = "single_download" #@param ["single_download", "batch_download","add","delete"]

%cd PornHub-downloader-python

if option == 'single_download':
  !python3 phdler.py custom "$pornhub_url"

elif option == 'add':
  !python3 phdler.py add "$pornhub_url"

elif option == 'delete':
  !python3 phdler.py delete "$pornhub_url"

else:
  !python3 phdler.py custom batch 

## <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="30" alt="Youtube-DL"/>

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute YouTube-DL
# @markdown 📝 Note: if you want to change an archive file just run this cell again.
Archive = False #@param {type:"boolean"}
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

## <img src='https://biplobsd.github.io/RLabClone/img/title_cloud_commander.png' height="30" alt="cloudCommander"/>


In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>⬅️ Install Cloud Commander (file manager)</h3>
#@markdown <br><center><img src='https://biplobsd.github.io/RLabClone/img/title_cloud_commander.png' height="30" alt="netdata"/></center>
# ================================================================ #
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]

import os, psutil, time, urllib.request
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    loadingAn,
    PortForward_wrapper,
)

loadingAn()
if os.path.isfile("/tools/node/bin/cloudcmd") == False:
  get_ipython().system_raw("npm cache clean -f && npm install -g n && n stable && npm i cloudcmd -g --force")

try:
  urllib.request.urlopen('http://localhost:7007')
except urllib.error.URLError:
  !nohup cloudcmd --online --no-auth --show-config --show-file-name \
        --editor 'deepword' --packer 'tar' --port 7007 \
        --no-confirm-copy --confirm-move --name 'File Manager' \
        --keys-panel --no-contact --console --sync-console-path \
        --no-terminal --no-vim --columns 'name-size-date' --no-log &
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['netdata', 19999, 'http'], 
                            ['cloudcmd', 7007, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/netdataCloudcmd.yml", 7044]
).start('cloudcmd')

## <img src='https://raw.githubusercontent.com/filebrowser/logo/master/icon_raw.svg' height="25" alt="filebrowser" /> ___Filebrowser___

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/filebrowser/logo/master/banner.svg' height="30" alt="filebrowser"/></center>
#@markdown <center><h3>filebrowser<br />📂 Web File Browser which can be used as a middleware or standalone app</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# OUTPUT_DIR = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess
)

loadingAn()

os.makedirs('tools/filebrowser/', exist_ok=True)

get_ipython().system_raw(r"curl -fsSL https://raw.githubusercontent.com/filebrowser/get/master/get.sh | bash")
if not findProcess("filebrowser", "--noauth"):
  runSh("filebrowser --noauth -r /content/ -p 4001 -d tools/filebrowser/filebrowser.db &", shell=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rclone', 5572, 'http'],
                            ['filebrowser', 4001, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/filebrowserRclone.yml", 4099]
).start('filebrowser')


## <img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2cli.png' height="30" alt="aira2"/> <font size=1px >___Direct link downloader___</font>

___Auto share link supported : Disk.yandex.com, Gdrive and youtube-dl all extractor...___

In [ ]:
URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

## <img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="30" alt="spotify-downloader" /> spotify-downloader

> Indented block



In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="40" alt="spotify-downloader"/></center>
#@markdown <center><h3>spotify-downloader<br />Download Spotify playlists from YouTube with albumart and meta-tags</h3></center><br>

import os, uuid, urllib.parse, re
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files


os.makedirs('tools/spotify-downloader/', exist_ok=True)
os.makedirs('downloads', exist_ok=True)
LATEST_VERSION = False #@param {type:"boolean"}

SET_VERSION = "2.0.8" #@param {type:"string"}
#@markdown  <font size='2px'>`https://pypi.org/project/spotdl/#history`</font>
# # Config files
# data = """spotify-downloader:
#   avconv: false
#   download-only-metadata: false
#   dry-run: false
#   file-format: '{artist} - {track_name}'
#   folder: /home/user/Music
#   input-ext: .m4a
#   log-level: INFO
#   manual: false
#   music-videos-only: false
#   no-fallback-metadata: false
#   no-metadata: false
#   no-spaces: false
#   output-ext: .mp3
#   overwrite: prompt
#   search-format: '{artist} - {track_name} lyrics'
#   skip: null
#   spotify_client_id: 4fe3fecfe5334023a1472516cc99d805
#   spotify_client_secret: 0f02b7c483c04257984695007a4a8d5c
#   trim-silence: false
#   write-successful: null
#   write-to: null
#   youtube-api-key: null
# """
# with open('tools/spotify-downloader/config.yml', 'w') as wnow:
#   wnow.write(data)

Links = widgets.Textarea(placeholder='''Link list
(one link per line)''')

fileFormat = widgets.Text(
    value='{artist} - {track_name}',
    placeholder='File name format',
    description="""File Name : file format to save the downloaded track with, each
                        tag is surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

searchFormat = widgets.Text(
    value='{artist} - {track_name} lyrics',
    placeholder='Search format',
    description="""Search Format : search format to search for on YouTube, each tag is
                        surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

tab = widgets.Tab()

LinksType = widgets.RadioButtons(
    options=['Songs', 'Playlist', 'Album', 'Username', 'Artist'],
    value='Songs',
    layout={'width': 'max-content'},
    description='Links type:',
    disabled=False,
)

SavePathYT = widgets.Dropdown(options=["/content/downloads", "/content"])

Extension = widgets.Select(options=["flac", "mp3", "m4a"], value="mp3")

TrimSilence = widgets.ToggleButton(
    value=False,
    description='Trim silence',
    disabled=False,
    button_style='',
    tooltip='remove silence from the start of the audio',
    icon='check'
)

writeM3u = widgets.ToggleButton(
    value=False,
    description='Write .m3u playlist',
    disabled=False,
    button_style='',
    tooltip='''generate an .m3u playlist file with youtube links
               given a text file containing tracks''',
    icon='check'
)

noMeta = widgets.ToggleButton(
    value=False,
    description='No metadata',
    disabled=False,
    button_style='',
    tooltip='do not embed metadata in tracks',
    icon='check'
)

noEnco = widgets.ToggleButton(
    value=False,
    description='No Encoding',
    disabled=False,
    button_style='',
    tooltip='do not encode media using FFmpeg',
    icon='check'
)

overwrite = widgets.Dropdown(
    disabled=False,
    options=['prompt', 'force', 'skip'],
    value='skip',
    description='Overwrite:',
)

quality = widgets.Dropdown(
    disabled=False,
    options=['worst', 'best'],
    value='best',
    description='Quality:',
)

inputExt = widgets.Dropdown(
    disabled=False,
    options=['automatic', 'm4a', 'opus'],
    value='automatic',
    description='Input format:',
)

logLevel = widgets.Dropdown(
    disabled=False,
    options=['INFO', 'WARNING', 'ERROR', 'DEBUG'],
    value='INFO',
    description='Log level:',
)

nf = widgets.ToggleButton(
    value=False,
    description='No fallback metadata',
    disabled=False,
    button_style='',
    tooltip='''do not use YouTube as fallback for metadata if track
                        not found on Spotify''',
    icon='check'
)

dryRun = widgets.ToggleButton(
    value=False,
    description='Dry run',
    disabled=False,
    button_style='',
    tooltip=''' show only track title and YouTube URL, and then skip
                        to the next track (if any)''',
    icon='check'
)

MusicVidOnly = widgets.ToggleButton(
    value=False,
    description='Music Videos Only',
    disabled=False,
    button_style='',
    tooltip='''search only for music videos on Youtube (works only
                        when YouTube API key is set''',
    icon='check'
)

NoSpaces = widgets.ToggleButton(
    value=False,
    description='No Spaces',
    disabled=False,
    button_style='',
    tooltip='''replace spaces with underscores in file names''',
    icon='check'
)

manual = widgets.ToggleButton(
    value=False,
    description='manually',
    disabled=False,
    button_style='',
    tooltip='''choose the track to download manually from a list of
                        matching tracks''',
    icon='check'
)

nr = widgets.ToggleButton(
    value=False,
    description='Keep original',
    disabled=False,
    button_style='',
    tooltip='''do not remove the original file after conversion''',
    icon='check'
)

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content/downloads", "/content"]


def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  mainTab = widgets.Box([
                        widgets.HBox([widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      LinksType, searchFormat, fileFormat, 
                                      widgets.HBox([TrimSilence, writeM3u, noMeta]),
                                      widgets.HBox([nf, dryRun, MusicVidOnly]),
                                      widgets.HBox([NoSpaces, manual, nr]),
                                      widgets.HBox([noEnco])
                                      ]),
                        widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), 
                                      Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), 
                                      ExtraArg, overwrite, quality, inputExt, 
                                      logLevel])
                        ])])
  
  tab.children = [mainTab]
  tab.set_title(0, 'spotify-downloader')
  display(HTML("<style>widget-tab-bar {display: none;}</style>"))
  display(tab)
  display(HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"), SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    
    if writeM3u.value:
      M3u = '--write-m3u'
    else:
      M3u = ''
    
    if TrimSilence.value:
      trmS = '--trim-silence'
    else:
      trmS = ''
    
    if noMeta.value:
      noM = '--no-metadata'
    else:
      noM = ''
    
    if noEnco.value:
      noE = '--no-encode'
    else:
      noE = ''

    if not overwrite.disabled:
      oW = f'--overwrite {overwrite.value}'
    else:
      oW = ''

    if not quality.disabled:
      qL = f'--quality {quality.value}'
    else:
      qL = ''
    
    if not inputExt.disabled:
      iE = f'--input-ext {inputExt.value}'
    else:
      iE = ''

    if not logLevel.disabled:
      lL = f'--log-level {logLevel.value}'
    else:
      lL = ''

    if nf.value:
      nfv = '--no-fallback-metadata'
    else:
      nfv = ''
    
    if dryRun.value:
      drR = '--dry-run'
    else:
      drR = ''
    
    if MusicVidOnly.value:
      MsV = '--music-videos-only'
    else:
      MsV = ''
    
    if NoSpaces.value:
      NoS = '--no-spaces'
    else:
      NoS = ''
    
    if manual.value:
      mal = '--manual'
    else:
      mal = ''
    
    if nr.value:
      nro = '--no-remove-original' 
    else:
      nro = ''
    
    if not searchFormat.value == '{artist} - {track_name} lyrics':
      seFor = f'--search-format "{searchFormat.value}"'
    else:
      seFor = ''
    
    if not fileFormat.value == '{artist} - {track_name}':
      fiFor = f'--file-format "{fileFormat.value}"'
    else:
      fiFor = ''
    
    if not LinksType.value == 'Songs':
      with open('tools/spotify-downloader/finish.txt', 'a+') as master:
        for Link in Links.value.splitlines():
          if LinksType.value == 'Playlist':
            outFileName = !spotdl --playlist $Link
          elif LinksType.value == 'Album':
            outFileName = !spotdl --album $Link
          elif LinksType.value == 'Username':
            outFileName = !spotdl -u $Link
          elif LinksType.value == 'Artist':
            outFileName = !spotdl --all-albums $Link
          filename = re.search(r"to\s(.+\.txt)", outFileName[-1]).group(1)
          with open(filename, 'r') as r:
            master.write(r.read())
    else:
      for Link in Links.value.splitlines():
        with open('tools/spotify-downloader/finish.txt', 'w') as master:
          master.write(Link)
    # Extra Arguments
    
    extraargC = ExtraArg.value
    cmd = r"spotdl -l 'tools/spotify-downloader/finish.txt' " \
      fr"-f '{SavePathYT.value}' " \
      fr"-o {Extension.value} " \
      f"{seFor} {fiFor} " \
      f"{M3u} {trmS} {oW} {noM} {noE} {nfv} {drR} {MsV} {NoS} {mal} {nro}" \
      f"{qL} {iE} {lL}" 
    !$cmd
    ShowYT()

if not os.path.isfile("/usr/local/bin/spotdl"):
  get_ipython().system_raw(f"pip3 install spotdl{'=='+SET_VERSION if LATEST_VERSION else ''} && apt -y install ffmpeg")

ShowYT()

## <img src='https://szyha.github.io/RcloneLabArchive/img/title_jdownloader.png' height="30" alt="JDownloader"/>

In [ ]:
#@markdown <h2>⬅️ Click Here to Install JDownloader</h2>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Dropdown(value="/content", options=["/content", "/content/Downloads"], description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/Shared drives/*")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"]
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output()
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath)
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output()
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output()
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output()
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output()
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output()
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output()
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-8-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()

In [ ]:
#@markdown <h3>⬅️ Click Here to Mount Google Drive</h3>
#@markdown <h4>⚠️ If you download the file directly to Google Drive, it might have data corruption, use this at your own risk!</h4>
#@markdown <h4>✅ The rclone method is recommended and always the safest.</h4>

import os, uuid; import ipywidgets as widgets
from IPython.display import clear_output
from google.colab import drive, output

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)
    
def DemountForm():
  clear_output(wait=True)
  display(MakeLabel("Demount Confirm?", ""), MakeButton("Confirm", Demount, "danger"), MakeButton("Cancel", ShowGD, "warning"))
    
def Demount():
  clear_output(wait=True)
  get_ipython().system_raw("pkill -9 -e -f drive")
  !rm -rf /root/.config/Google/DriveFS/
  !echo "If you want to mount Google Drive run this cell again."
  
def ShowGD():
  try:
    RefreshPath()
  except:
    clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), MakeButton("Demount", DemountForm, "danger"))

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output(wait=True)
    
ShowGD()

## <img src='https://i.imgur.com/ql7GYNT.png' height="30" alt="torrentrrent"/> __Cloud torrent__

In [ ]:
#@markdown <h2>⬅️ Click Here to Install Cloud Torrent</h2>
import string, os, urllib.request, json, time, getpass; from IPython.display import clear_output

if os.path.isfile("/usr/local/bin/cloud-torrent") == False:
 	!curl https://i.jpillora.com/cloud-torrent! | bash
else:
  !pkill cloud-torrent
clear_output(wait=True)

if os.path.isfile("/usr/local/bin/ngrok") == False:
  !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -qq -n ngrok-stable-linux-amd64.zip
  !mv ngrok /usr/local/bin/ngrok
  !rm -f /content/ngrok-stable-linux-amd64.zip
  clear_output(wait=True)
else:
  !pkill ngrok

if os.path.isfile("/usr/bin/rclone") == False:
  get_ipython().system_raw("rm -rf /content/sample_data/ && curl -s https://rclone.org/install.sh | sudo bash")
else:
  clear_output(wait=True)

print("Copy authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()
get_ipython().system_raw('/usr/local/bin/ngrok authtoken $authtoken && /usr/local/bin/ngrok tcp 4444 &')

#Opening cloud-torrent in background
!nohup cloud-torrent --port 4444 --title "Cloud Torrent" --log  &
clear_output()

#Get public address and print connect command
time.sleep(10)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'Open this link in new tab http://{host}:{port}')

## <img src='https://dominicm.com/wp-content/uploads/2017/03/rtorrent-arch-linux-450x450.png' height="30" alt="rTorrent"/> __rTorrent__

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/Flood-UI/flood/master/flood.png' height="100" alt="flood"/></center>
#@markdown <center>Flood-UI<h3>A web UI for rTorrent with a Node.js backend and React frontend.</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from shutil import copyfile
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")


from ttmg import (
    updateCheck,
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    textAn
)

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()
loadingAn()

os.makedirs('tools/', exist_ok=True)
os.makedirs("/content/downloads", mode=0o775, exist_ok=True)
os.makedirs("/content/tools/rtorrent/session", mode=0o775, exist_ok=True)

configData = """
# Where rTorrent saves the downloaded files
directory = /content/downloads

# Where rTorrent saves the session
session = /content/tools/rtorrent/session

# Which ports rTorrent can use (Make sure to open them in your router)
port_range = 50000-50000
port_random = no

# Check the hash after the end of the download
check_hash = yes

# Enable DHT (for torrents without trackers)
dht = auto
dht_port = 6881
peer_exchange = yes

# Authorize UDP trackers
use_udp_trackers = yes

# Enable encryption when possible
encryption = allow_incoming,try_outgoing,enable_retry

# SCGI port, used to communicate with Flood
scgi_port = 127.0.0.1:5000
"""
with open("/root/.rtorrent.rc", 'w') as rC:
  rC.write(configData)

if not os.path.exists("/content/tools/flood/config.js"):
  runSh("apt install rtorrent screen mediainfo -y")
  runSh("git clone --depth 1 https://github.com/jfurrow/flood.git tools/flood",
        shell=True)
  copyfile("tools/flood/config.template.js", "tools/flood/config.js")
  runSh("npm install", shell=True, cd="tools/flood/")
  runSh("npm install pm2 -g", shell=True, cd="tools/flood/")
  runSh("npm run build", shell=True, cd="tools/flood/")

  userDB = r"""{"username":"admin","password":"$argon2i$v=19$m=4096,t=3,p=1$3hJdjMSgwdUnJ86uYBhOnA$dud5j5/IokJ3hyb+v5aqmDK0jwP9X5W2pz6Qqek++Tk","host":"127.0.0.1","port":"5000","isAdmin":true,"_id":"jLJcPySMAEgp35uB"}
{"$$indexCreated":{"fieldName":"username","unique":true,"sparse":false}}
"""
  userSettingsDB = r"""{"id":"startTorrentsOnLoad","data":true,"_id":"5leeeHwIN9rKLgG9"}
{"id":"torrentListColumnWidths","data":{"sizeBytes":61,"ratio":56,"peers":62},"_id":"PnB52rZSPg5fLEN9"}
{"id":"torrentDestination","data":"/content/downloads","_id":"YcGroeyigKYWM8Ol"}
{"id":"mountPoints","data":["/"],"_id":"gJlGwWqOsyPfkLyJ"}
{"id":"torrentListViewSize","data":"expanded","_id":"q0CmirE9c0KnDGV3"}
"""

  os.makedirs("tools/flood/server/db/jLJcPySMAEgp35uB/settings", exist_ok=True)
  with open("tools/flood/server/db/users.db", 'w') as wDB:
    wDB.write(userDB)
  with open("tools/flood/server/db/jLJcPySMAEgp35uB/settings/settings.db", 'w') as wDB:
    wDB.write(userSettingsDB)

if not findProcess("rtorrent", ""):
  runSh("screen -d -m -fa -S rtorrent rtorrent", shell=True)
if not findProcess("node", "start.js"):  
  runSh("pm2 start server/bin/start.js", shell=True, cd="tools/flood/")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()

PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['flood', 3000, 'http']], 'ap', 
    [f"{HOME}/.ngrok2/flood.yml", 1463]
).start('flood', btc='b', displayB=True)

## <img src='https://szyha.github.io/RcloneLabArchive/img/title_netdata.png' height="30" alt="netdata"/>


In [ ]:
# ============================= FORM ============================= #
#@markdown <br><center><img src='https://szyha.github.io/RcloneLabArchive/img/title_netdata.png' height="30" alt="netdata"/></center>
#@markdown <center><h3>Real-time Server Monitoring</h3></center><br>
# ============================= FORM ============================= #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
# ============================= FORM ============================= #
# ================================================================ #

import os, psutil, subprocess, shlex
from IPython.display import HTML, clear_output
import time
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py" \
                f" -O {HOME}/.ipython/ttmg.py"
    subprocess.run(shlex.split(shellCmd))

from ttmg import (
    loadingAn,
    PortForward_wrapper,
    runSh,
    textAn
)

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def Start_ServerMT():
  if CheckProcess("netdata", "") != True:
    runSh('/usr/sbin/netdata', shell=True)

loadingAn()      

if not os.path.isfile("/usr/sbin/netdata"):
  clear_output(wait=True)
  textAn("[Installing] Wait for almost 2 minutes...")
  #start = time.time()
  get_ipython().system_raw("bash <(curl -Ss https://my-netdata.io/kickstart.sh) --dont-wait --dont-start-it")
  #end = time.time()
  clear_output(wait=True)
Start_ServerMT()

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['netdata', 19999, 'http'], 
                            ['cloudcmd', 7007, 'http']], 'au', 
    [f"{HOME}/.ngrok2/netdataCloudcmd.yml", 7044]
).start('netdata', 'g')

## <img src='https://yuju.pw/y/qVwA.png' height="30" alt="RcloneLab"/> ___SSH Access___


In [ ]:
#@markdown <h3><b>←</b> Run to enable SSH acccess
#@markdown <h5><i>This will ask for your ngrok token from:</i> <a href="https://dashboard.ngrok.com/auth">https://dashboard.ngrok.com/auth</a></h5>
#Generate root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc
#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')
#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()
#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## <img src='https://jellyfin.org/favicon.ico' height="30" alt="V2Ray" /> __Jellyfin__

In [ ]:
#@markdown <br><center><img alt="" src="..." height="50" alt="jellyfin"/></center>
#@markdown <center><h3>Jellyfin<br />The Free Software Media System.</h3></center><br>
import os, zipfile, tarfile, requests, re
import urllib.request
from IPython.display import clear_output, display
from ipywidgets import Dropdown, Select
from subprocess import Popen
import xml.etree.ElementTree as ET


#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
LATEST_VERSION = False  # @param {type:"boolean"}
CUSTOM_VERSION = "10.7.6"  # @param {type:"string"} 
#@markdown <small>&emsp;Version are available on https://repo.jellyfin.org/archive/linux/stable/</small>
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : admin

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    CWD,
    textAn
)

loadingAn()

os.makedirs('tools/jellyfin/temp', exist_ok=True)
os.makedirs('tools/jellyfin/transcodes', exist_ok=True)
os.makedirs('downloads', exist_ok=True)

def ff_loader(name):
  import requests

  url = requests.get(
    "https://ffbinaries.com/api/v1/version/latest"
    ).json()['bin']['linux-64'][name]
  urllib.request.urlretrieve(url, f'tools/jellyfin/L{name}.zip')

  with zipfile.ZipFile(f'tools/jellyfin/L{name}.zip', 'r') as zip_ref:
      zip_ref.extractall('tools/jellyfin/')
  os.remove(f"tools/jellyfin/L{name}.zip")
  runSh("rm -rf tools/jellyfin/__MACOSX")
  os.replace(f"tools/jellyfin/{name}", f"/usr/bin/{name}")
  os.chmod(f"/usr/bin/{name}", 0o755)

def customVersionSelect():
  allVersionPage = requests.get("https://repo.jellyfin.org/archive/linux/stable/").text
  allVersinoTags = re.findall(r'">(\d+\.\d+\.\d+)', allVersionPage, re.M)
  option_list = sorted(allVersinoTags, reverse=True)
  dropdown = Select(description="Choose one:", options=option_list)
  dropdown.observe(main, names='value')
  display(dropdown)

def startService():
  # Starting jellyfin
  if not findProcess("jellyfin", "--ffmpeg"):
    CM = r"tools/jellyfin/jellyfin/jellyfin --service " \
    f"-C tools/jellyfin -d {CWD}/tools/jellyfin"
    Popen(CM.split())


  # START_SERVER
  # Ngrok region 'us','eu','ap','au','sa','jp','in'
  Server = PortForward_wrapper(
      PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['jellyfin', 8096, 'http']],
      REGION.lower(), [f"{HOME}/.ngrok2/jellyfin.yml", 7097]
  ).start('jellyfin', displayB=True)

def main(version=None):

  # JellyfinMain
  if LATEST_VERSION:
    serverUrl = "https://repo.jellyfin.org/releases/server/linux/stable/combined/"
  elif CUSTOM_VERSION:
    serverUrl = f"https://repo.jellyfin.org/archive/linux/stable/{CUSTOM_VERSION}/combined/"
  else:
    serverUrl = f"https://repo.jellyfin.org/archive/linux/stable/{version.new}/combined/"

  server = requests.get(serverUrl).text
  server_file = re.search(r'f=\"(.+)\">jellyfin_(.+)\.tar\.gz<',
                              server, re.MULTILINE)
  server_file_url = server_file.group(1)
  versionTag = server_file.group(2)

  clear_output(True)
  loadingAn()
  textAn(f"Installing v{versionTag} ...", ty='twg')
  
  outputFile = 'tools/jellyfin/temp/jellyfin.tar.gz'
  urllib.request.urlretrieve(serverUrl+server_file_url, outputFile)
  with tarfile.open(outputFile, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/jellyfin')
  os.renames(f'tools/jellyfin/jellyfin_{versionTag}',
            'tools/jellyfin/jellyfin')
  os.remove(outputFile)

  # Default config data
  defaultConfig = "https://raw.githubusercontent.com/biplobsd" \
  "/OneClickRun/master/res/appdata/jellyfin/data.tar.gz"
  outputFile = 'tools/jellyfin/temp/data.tar.gz'
  urllib.request.urlretrieve(defaultConfig, outputFile)
  with tarfile.open(outputFile, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/jellyfin')
  os.remove(outputFile)
  
  # https://github.com/rokibulislaam/colab-ffmpeg-cuda
  ## with GPU 
  if os.system("nvidia-smi --query-gpu=name --format=csv,noheader")==0: 
    encoConfigP = 'tools/jellyfin/config/encoding.xml'
    mytree = ET.parse(encoConfigP) 
    myroot = mytree.getroot() 

    def addnew(name, value):
      ET.SubElement(myroot, name).text = value

    addnew('TranscodingTempPath', '/content/tools/jellyfin/transcodes')
    addnew('HardwareAccelerationType', 'nvenc')
    addnew('DeinterlaceMethod', 'yadif')
    addnew('EnableDecodingColorDepth10Hevc', 'true')
    addnew('EnableDecodingColorDepth10Vp9', 'true')
    addnew('EnableDecodingColorDepth10Vp9', 'true')
    addnew('EncoderPreset', 'slower')
    addnew('EncodingThreadCount', '0')

    for tagname in myroot.iter('HardwareDecodingCodecs'):
      ET.SubElement(tagname, 'string').text = 'hevc'
      ET.SubElement(tagname, 'string').text = 'mpeg2video'
      ET.SubElement(tagname, 'string').text = 'mpeg4'
      ET.SubElement(tagname, 'string').text = 'vp8'
      ET.SubElement(tagname, 'string').text = 'vp9'

    mytree.write(encoConfigP) 

    runSh(
        "git clone --depth 1 --branch main https://github.com/rokibulislaam/colab-ffmpeg-cuda.git tools/colab-ffmpeg-cuda")
    runSh("cp -r tools/colab-ffmpeg-cuda/bin/. /usr/bin/")
    ffmpegPath = '/usr/bin/ffmpeg'
  else:
    ## without GPU
    # ffmpeg
    ff_loader('ffmpeg')

    # ffprobe
    ff_loader('ffprobe')
    
    ffmpegPath = '/usr/bin/ffmpeg'
  
  startService()

if not os.path.exists("tools/jellyfin/jellyfin/jellyfin"):
  if LATEST_VERSION or CUSTOM_VERSION:
    main()
  else:
    customVersionSelect()
else:
  startService()

## <img src='https://rclone.org/img/rclone-32x32.png' height="30" alt="rclone"/><i>__Autorclone__</i>
<h6> ✧  Break The 750GB Limit</h6>


In [ ]:
#@markdown <h3><b>←</b> Run this cell to install all required modules</h3>

!curl https://rclone.org/install.sh | sudo bash
###
%cd /content/
!git clone https://github.com/totalleecher/AutoRclone.git
###
%cd /content/AutoRclone
!sudo pip3 install -r requirements.txt

from IPython.display import HTML, clear_output
clear_output()
print("Successfully Installed")

In [ ]:
#@markdown <h3><b>←</b> Enable Drive Api & Get credentials.json</h3>
!python3 gen_sa_accounts.py --quick-setup -1

from IPython.display import Image, display
display(Image(url = "https://raw.githubusercontent.com/totalleecher/Codemaster/master/Screenshot_2020-03-24%20Python%20Quickstart%20Google%20Drive%20API%20Google%20Developers.png"))

In [ ]:
#@markdown <h3><b>←</b> Upload "credentials.json"</h3>

from google.colab import files
uploaded = files.upload()

In [ ]:
#@markdown <h3><b>←</b> Generate Service Accounts (if you already have service account please skip this step)</h3>

!python3 gen_sa_accounts.py --quick-setup -1

### Method 1 (Adding Services Acc to **Team Drive**)

In [ ]:
#@markdown <h3><b>←</b> Run this cell to add all SAs to a Team Drive</h3>
#@markdown </h5>Team Drive ID (TDid) = https://drive.google.com/drive/folders/<code>all-of-these-letters-numbers-and-symbols </code></h5>
#@markdown <h5><i>This may take some time. Be patient!</i></h5> 
import os
TDid = "0AJ6E5j3WzLBQUk9PVA" #@param {type:"string"}
!python3 add_to_team_drive.py -d "$TDid"

In [ ]:
#@markdown <h3><b>←</b> **Final Step -** Running Autorclone</h3>
#@markdown <p>- Add the Source Folder ID and then the Destination Folder ID</p>
#@markdown <p>- Note: Your source*** *(*`src`*)* ***folder must be publically accessible***</p>

#@markdown Folder IDs are found at the end of the URL:
#@markdown ![](https://yuju.pw/y/qNvy.png)

#@title <h3><b>←</b> Run to clone all folders + files from <code>src</code> to <code>dest</code>
#@markdown <i>Paste only the Folder IDs for these values</i>:
source = "1vLss1ytzo3UZboHUoEtyuQJaLdeP2C1c" #@param {type:"string"}
destination = "0AJ6E5j3WzLBQUk9PVA" #@param {type:"string"}

!python3 rclone_sa_magic.py -s "$source" -d "$destination" -b 1 -e 600

### Method 2 (Adding Services Acc to **Google Group**)

In [ ]:
#@markdown Create Google Group then add the service accounts as members by hand. Limit is 10 at a time, 100 a day but if you read our warning and notes above, you would have 1 project and hence easily in your range.

import os
from IPython.display import HTML, clear_output

print ("https://groups.google.com/")

In [ ]:
#@markdown <h3><b>←</b> **Final Step -** Running Autorclone</h3>
#@markdown <p>- Add the Source Folder ID and then the Destination Folder ID</p>
#@markdown <p>- Note: ***Your source*** *(*`src`*)* ***folder must be publically accessible***</p>

#@markdown Folder IDs are found at the end of the URL:
#@markdown ![](https://yuju.pw/y/qNvy.png)

#@title <h3><b>←</b> Run to clone all folders + files from <code>src</code> to <code>dest</code>
#@markdown <i>Paste only the Folder IDs for these values</i>:
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}

!python3 rclone_sa_magic.py -s "$source" -d "$destination" -b 1 -e 600

### Extra (Checking File size & Qty)

In [ ]:
#@markdown <h3><b>←</b> Size of Source Directory</h3> 
!rclone --config rclone.conf size --disable ListR src001:

In [ ]:
#@markdown <h3><b>←</b> Size of Destination Directory</h3> 
!rclone --config rclone.conf size --disable ListR src001:

##<img src='https://www.gstatic.com/android/market_images/web/play_prism_hlock_2x.png' height="30" alt="7zip"><i>__Apk Downloader__</i>

In [ ]:
#@title ⬅️ Clone repo and install requirements - Need to restart runtime after installed

%cd /content/
!git clone https://github.com/HamidrezaMoradi/APK-Downloader.git
#####
%cd /content/APK-Downloader
!pip3 install -r requirements.txt 

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
!python3 APK-Downloader.py 

In [ ]:
#@markdown <h3>⬅️ Click Here to move downloaded apk files to mounted gdrive</h3>

import os

%cd /content/drive/My Drive
os.makedirs("Downloaded Apk", exist_ok=True)


#####
%cd /content/APK-Downloader/File

!mv *.apk "/content/drive/My Drive/Downloaded Apk"

from IPython.display import HTML, clear_output

clear_output()

print("Files Successfully Moved")


##<img src='https://yuju.pw/y/qUkg.png' height="30" alt="7zip"><i>__7-Zip + UNRAR__</i>

In [ ]:
#@markdown <h3><b>←</b> <i>Required:</i> Install 7-Zip + UNRAR
!sudo apt update
!sudo apt install p7zip-full p7zip-rar unrar rar

In [ ]:
#@markdown <h3>» Run to change current working directory
import os
new_dir = "" #@param {type:"string"}

%cd "$new_dir"

In [ ]:
#@title » Extract Archive (<i>.zip, .rar, .7z, etc.</i>)
import os, sys, re

archive_file_path = "" #@param {type:"string"}
os.environ['inputFile'] = archive_file_path

!7z e "$inputFile"

In [ ]:
#@title » Create .zip .7z from Folder
import os, sys, re

folder_path = "/content/drive/My Drive/Deepnude/first-order-motion-model" #@param {type:"string"}
archiving_method = 'zip' #@param ["zip", "7z" ]

output_file_path = re.search("^[\/].+\/", folder_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", folder_path)
folder_name = re.sub("^[\/]", "", delsplit.group(0))

os.environ['inputDir'] = folder_path
os.environ['outputPath'] = output_file_path_raw
os.environ['folderName'] = folder_name

if archiving_method == 'zip':
   !7z a -tzip "$outputPath"/"$folderName".zip "$inputDir"

if archiving_method == '7z':
   !7z a -t7z -m0=lzma -mx=9 -mfb=258 -md=32m -ms=on "$outputPath"/"$folderName".7z "$inputDir"

## <img src='https://avatars1.githubusercontent.com/u/223312?s=400&v=4' height="50" alt="colab" />  __Transmission__



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>


!apt-get install transmission-cli
from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add From Magnet Link or .torrent Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want</h5><br>


import os
torrent_URL = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}

!transmission-cli -w "$output_dir" "$torrent_URL"



## <img src='https://github.com/ronggang/transmission-web-control/raw/master/src/tr-web-control/logo.png' height="55" alt="Transmission-Web-Control"/> 

In [ ]:
# ============================= FORM ============================= #
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}

# ================================================================ #

import os, psutil, time, urllib.request, pathlib
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)


loadingAn()
if not os.path.exists('/usr/bin/transmission-daemon'):
  os.makedirs('downloads', exist_ok=True)
  os.makedirs('tools/transmission/', exist_ok=True)
  runSh('apt install transmission-daemon')
  nTWC = "https://raw.githubusercontent.com/ronggang/" \
    "transmission-web-control/master/release/install-tr-control.sh"
  urllib.request.urlretrieve(nTWC, 'tools/transmission/trInstall.sh')
  runSh('bash tools/transmission/trInstall.sh auto')
  try:
    pathlib.Path('tools/transmission/trInstall.sh').unlink()
  except FileNotFoundError:
    pass

if not findProcess('transmission-daemon', '--no-watch-dir'):
  !transmission-daemon --no-watch-dir --config-dir tools/transmission \
    --port 9091 --download-dir /content/downloads/ --dht --utp --no-portmap \
    --peerlimit-global 9999 --peerlimit-torrent 9999 --no-global-seedratio \
    -u admin -v admin --auth


# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['vuze', 9595, 'http'], 
                            ['transmission', 9091, 'http']], 'sa', 
    [f"{HOME}/.ngrok2/transmissionVuze.yml", 4058]
).start('transmission', displayB=False)
displayUrl(server, pNamU='Transmission : ', 
           ExUrl=fr"http://admin:admin@{server['url'][7:]}",
           btc='r')

## <img src='https://upload.wikimedia.org/wikipedia/commons/9/9f/UTorrent_%28logo%29.png' height="25" alt="µTorrent" /> ___µTorrent___

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/9/9f/UTorrent_%28logo%29.png' height="30" alt="µTorrent"/></center>
#@markdown <center><h3>µTorrent<br />uTorrent is a proprietary adware BitTorrent client owned and developed by BitTorrent, Inc.</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
# OUTPUT_DIR = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")
r = get_ipython()
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")


from ttmg import (
    updateCheck,
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl
)

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()
loadingAn()

#uTorrent installing...
if not os.path.exists("/usr/bin/utserver"):
  os.makedirs("downloads", exist_ok=True)
  r.system_raw("apt install libssl1.0.0 libssl-dev")
  r.system_raw(r"wget http://download-new.utorrent.com/endpoint/utserver/os/linux-x64-ubuntu-13-04/track/beta/ -O utserver.tar.gz")
  r.system_raw(r"tar -zxvf utserver.tar.gz -C /opt/")
  r.system_raw("rm -f utserver.tar.gz")
  r.system_raw("mv /opt/utorrent-server-* /opt/utorrent")
  os.chmod("/opt/utorrent", 0o777)
  r.system_raw("ln -s /opt/utorrent/utserver /usr/bin/utserver")
  urllib.request.urlretrieve(
      "https://raw.githubusercontent.com/helloW3c/Google-Colab-CloudTorrent/master/res/utorrent/utserver.conf",
      "/opt/utorrent/utserver.conf")
if not findProcess("utserver", "-settingspath"):
  cmd = "utserver -settingspath /opt/utorrent/" \
        " -configfile /opt/utorrent/utserver.conf" \
        " -daemon"
  runSh(cmd, shell=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['deluge', 8112, 'http'],
                            ['utorrent', 5454, 'http']], 'eu', 
    [f"{HOME}/.ngrok2/delugeUtorrent.yml", 4042]
).start('utorrent', displayB=False)
displayUrl(Server, pNamU='uTorrent : ', 
           ExUrl=fr"http://admin:admin@{Server['url'][7:]}/gui",
           btc="g")



## <img src='https://i.imgur.com/DOoxSuO.png' height="30" alt="Simple Torrent" />



In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://i.imgur.com/DOoxSuO.png' height="50" alt="netdata"/></center>
#@markdown <center><h3>SimpleTorrent is a a self-hosted remote torrent client.</h3></center><br>
#@markdown <center><h5>Auto upload to gdrive after task completed. if gdrive is mounted and if not working tick install_old_version.</h5></center><br>

import os
import time
import pathlib 
import urllib.request
from IPython.display import clear_output

# script version
Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
Install_old_version = True  # @param {type:"boolean"}
Auto_UP_Gdrive = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
AUTO_MOVE_PATH = "/content/drive/Shareddrives"  # @param {type:"string"}
force_change_version = "" # https://github.com/boypt/simple-torrent/releases
rclone_DestinationPath = "" # @param {type:"string"}
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    PortForward_wrapper
)
# making enviroment for simple-torrent
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('torrents').mkdir(mode=0o777, exist_ok=True)

def generateCmd(src, dst):
  FAST_LIST = True
  PATH_RClone_Config = "/usr/local/sessionSettings"
  cmd = f'rclone move "{src}" "{dst}" ' \
    f'--config {PATH_RClone_Config}/rclone.conf ' \
    f'{"--fast-list" if FAST_LIST else ""} --user-agent "Mozilla" ' \
    '--transfers 20 --checkers 20 --drive-server-side-across-configs ' \
    '-c --buffer-size 256M --drive-chunk-size 256M ' \
    '--drive-upload-cutoff 256M --drive-acknowledge-abuse ' \
    '--drive-keep-revision-forever --tpslimit 95 --tpslimit-burst 40 ' \
    '--stats-one-line --stats=5s -v'
  return cmd


if Auto_UP_Gdrive:
  # hCode1 = "https://raw.githubusercontent.com/helloW3c/" \
  #           "Google-Colab-CloudTorrent/master/res/scripts/doneCMD.sh"
  # urllib.request.urlretrieve(hCode1, f'{HOME}/doneCMD.sh')
  data = """#!/bin/bash

dir=${CLD_DIR}
path=${CLD_PATH}
abp="${dir}/${path}"
type=${CLD_TYPE}

if [[ ${type} == "torrent" ]]; then
"""
  nUpload = """    
    
    #Upload to Gdrive
    #mkdir -p "%s/$(dirname "${path}")"
    mv "${abp}" "%s/${path}"

    """ % (AUTO_MOVE_PATH, AUTO_MOVE_PATH)
  
  rcloneUpload = """
    #You can also use rcone move file to remote
    %s
    """ % generateCmd(r"${abp}", rclone_DestinationPath)
  
  end = """
fi
"""
  data = data + (rcloneUpload if rclone_DestinationPath else nUpload) + end
  with open(pathDoneCMD, 'w') as w:
    w.write(data)
  os.chmod(pathDoneCMD, 0o755)
else:
  try:
    os.unlink(pathDoneCMD)
  except FileNotFoundError:
    pass
  
configPath = pathlib.Path('cloud-torrent.json')
configsdata = r"""
{{
  "AutoStart": true,
  "EngineDebug": false,
  "MuteEngineLog": true,
  "ObfsPreferred": true,
  "ObfsRequirePreferred": false,
  "DisableTrackers": false,
  "DisableIPv6": false,
  "DownloadDirectory": "downloads/",
  "WatchDirectory": "torrents/",
  "EnableUpload": true,
  "EnableSeeding": false,
  "IncomingPort": 50007,
  "DoneCmd": "{}/doneCMD.sh",
  "SeedRatio": 1.5,
  "UploadRate": "High",
  "DownloadRate": "Unlimited",
  "TrackerListURL": "https://trackerslist.com/best.txt",
  "AlwaysAddTrackers": true,
  "ProxyURL": ""
}}
""".format(HOME)
with open(configPath, "w+") as configFile:
  configFile.write(configsdata)
#####################################

if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()

# Simple Torrent installing ...
loadingAn()
if not os.path.isfile("/usr/local/bin/cloud-torrent"):
    dcmd = "wget -qq https://raw.githubusercontent.com/helloW3c/" \
            "Google-Colab-CloudTorrent/master/res/scripts/" \
            "simpleCloudInstaller.sh -O /root/simpleCloudInstaller.sh"
    runSh(dcmd)
    if Install_old_version:
        runSh("bash /root/simpleCloudInstaller.sh 1.2.3")
    else:
        runSh("bash /root/simpleCloudInstaller.sh")

if force_change_version:
  runSh("pkill cloud-torrent")
  runSh(f"bash /root/simpleCloudInstaller.sh {force_change_version}")


#Opening cloud-torrent in background
if not findProcess("cloud-torrent", "cloud-torrent"):
  PORT = 4444
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    cmdC = f'cloud-torrent --port {PORT} ' \
        '-t "SimpleTorrent" ' \
        '-c cloud-torrent.json ' \
        '--host 0.0.0.0 --disable-log-time ' \
        '&'
    for run in range(10):    
      runSh(cmdC, shell=True)
      time.sleep(3)
      try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
        break
      except:
        print("Error: Simple-Torrent not starting. Retrying ...")

# START_SERVER
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['simple-torrent', 4444, 'http'], 
    ['peerflix-server', 4445, 'http']], 'us', 
    [f"{HOME}/.ngrok2/sTpS.yml", 4040]
).start('simple-torrent')

## <img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="30" alt="pyLoad" />

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="50" alt="pyLoad"/></center>
#@markdown <center><h3>pyLoad Webinterface<br />The free and open-source Download Manager written in pure Python</h3></center><br>
import os, pathlib, zipfile, re, threading
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess
)

loadingAn()

os.makedirs('downloads', exist_ok=True)
os.makedirs('tools/pyload', exist_ok=True)

# Downloading latest version of pyload
if not os.path.exists("tools/pyload/pyload-stable"):
  urlF = 'https://github.com/pyload/pyload/archive/stable.zip'
  conf = 'https://raw.githubusercontent.com/biplobsd/' \
          'OneClickRun/master/res/appdata/pyload/pyload.conf'
  db = 'https://github.com/biplobsd/OneClickRun/raw/master/' \
        'res/appdata/pyload/files.db'
  urllib.request.urlretrieve(urlF, 'tools/pyload.zip')
  urllib.request.urlretrieve(conf, 'tools/pyload/pyload.conf')
  urllib.request.urlretrieve(db, 'tools/pyload/files.db')
  with zipfile.ZipFile('tools/pyload.zip', 'r') as zip_ref:
      zip_ref.extractall('tools/pyload')
  try:
    pathlib.Path('tools/pyload.zip').unlink()
  except FileNotFoundError:
    pass

  # runSh("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev")
  # runSh("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr")

  # Using Threading to install
  threadRun = [
    threading.Thread(target=runSh, args=("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev",)),
    threading.Thread(target=runSh, args=("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr",))
  ]
  for perTh in threadRun:perTh.start()
  for perTh in threadRun:perTh.join()

if not findProcess("python2.7", "pyLoadCore.py"):
  runCmd = "python2.7 /content/tools/pyload/pyload-stable/pyLoadCore.py" \
          " --configdir=/content/tools/pyload" \
          " --no-remote" \
          " --daemon"
  runSh(runCmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfig', 53682, 'http'], 
                            ['pyload', 8000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/rclonePyload.yml", 4074]
).start('pyload')

## <img src='https://www.vozidea.com/wp-content/uploads/2018/05/ffmpeg-logo-grande.png ' height="30" alt="RcloneLab"/>    
***Required to use Scripts:*** Install FFmpeg, VCSI & Mkvtoolnix

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/cheems/FFmpeg-for-GDrive/master/res/ffmpeg.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Upgrade FFmpeg to v4.2.2</h3></center><br>
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cheems/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

##*Run FFmpeg Scripts (*Convert, Edit, Trim + more*)

### <font size="4">» </font><font size="5">Display Media File Metadata

In [ ]:
import os, sys, re

media_file_path = "" #@param {type:"string"}

os.environ['inputFile'] = media_file_path

!ffmpeg -i "$inputFile" -hide_banner

> *You can ignore the* "`At least one output file must be specified`" *error after running this.*




### <font size="4">» </font><font size="5">Convert *Video File* ➔ *.mp4* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mp4

### <font size="4">» </font><font size="5">Convert *Video File* ➔ *.mkv* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mkv

### <font size="4">» </font><font size="5">Merge *Video File* + *Subs File* ➔ *.mkv* (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
subs_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['inputSubsFile'] = subs_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -i "$inputSubsFile" -c:v copy -c:a copy "$outputPath"/"$fileName"-out.mkv

### <font size="4">» </font><font size="5">Trim Video File (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

### <font size="4">» </font><font size="5">Crop Video

<h3> Crop Variables Explanation:

* `out_width` = The width of your cropped video file.
* `out_height` = The height of your cropped video file.
* `starting_position_x` & `starting_position_y` = These values define the x & y coordinates of the top left corner of your original video to start cropping from.

###### *Example: For cropping the black bars from a video that looked like* [this](https://yuju.pw/y/312r.png):
* *For your starting coordinates* (`x` , `y`) *you would use* (`0` , `138`).
* *For* `out_width` *you would use* `1920`. *And for* `out_height` *you would use `804`.*





In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
out_width = "1920" #@param {type:"string"}
out_height = "804" #@param {type:"string"}
starting_position_x = "0" #@param {type:"string"}
starting_position_y = "138" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outWidth'] = out_width
os.environ['outHeight'] = out_height
os.environ['positionX'] = starting_position_x
os.environ['positionY'] = starting_position_y
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -filter:v "crop=$outWidth:$outHeight:$positionX:$positionY" "$outputPath"/"$fileName"-CROP."$fileExtension"

### <font size="4">» </font><font size="5">Extract Audio from Video File (*Lossless*)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_extension = 'm4a' #@param ["m4a", "mp3", "opus", "flac", "wav"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
output_file_path = re.search("^[\/].+\/", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = output_file_extension

!ffmpeg -hide_banner -i "$inputFile" -vn -c:a copy "$outputPath"/"$fileName"-audio."$fileType"

### <font size="4">» </font><font size="5">Re-encode a Video to a Different Resolution

In [ ]:
import os, sys, re

video_file_path = '' #@param {type:"string"}
resolution = '1080p' #@param ["2160p", "1440p", "1080p", "720p", "480p", "360p", "240p"]
file_type = 'mp4' #@param ["mkv", "mp4"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
testsplit = video_file_path.split("/")
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
resolution_raw = re.search("[^p]{3,4}", resolution)
output_file_path = re.search("^[\/].+\/", video_file_path)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = file_type
os.environ['resolutionHeight'] = resolution_raw.group(0)

!ffmpeg -hide_banner -i "$inputFile" -vf "scale=-1:"$resolutionHeight"" -c:a copy -strict experimental "$outputPath"/"$fileName"-"$resolutionHeight"p."$fileType"

### <font size="4">» </font><font size="5">Extract Individual Frames from Video

In [ ]:
#@markdown This will create a folder in the same directory titled "`Extracted Frames`"
* [*Example*](https://yuju.pw/y/36pP.png) *of output folder*
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}
frame_rate = "23.976" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['frameRate'] = frame_rate
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir "$outputPath"/"Extracted Frames"
!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -r "$frameRate"/1 "$outputPath"/"Extracted Frames"/frame%04d.png

### <font size="4">» </font><font size="5">Generate Thumbnails - Preview from Video (3x2)

In [ ]:
#@markdown Example of output image: https://yuju.pw/y/39i2.png <br>
import os, sys, re

video_file_path = "" #@param {type:"string"}
output_file_type = 'png' #@param ["png", "jpg"]

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['outputExtension'] = output_file_type
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -vframes 1 -q:v 2 -vf "select=not(mod(n\,200)),scale=-1:480,tile=3x2" -an "$outputPath"/"$fileName"_thumbnails."$outputExtension"

### <font size="4">» </font><font size="5">Convert Audio Filetype (*mp3, m4a, ogg, flac, etc.*)

In [ ]:
import os, sys, re

audio_file_path = "" #@param {type:"string"}
output_file_type = "mp3" #@param ["mp3", "ogg", "m4a", "opus", "flac", "alac", "wav"]

output_file_path = re.search("^[\/].+\/", audio_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", audio_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = audio_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileExtension'] = output_file_type
os.environ['fileName'] = filename_raw

!ffmpeg -hide_banner -i "$inputFile" "$outputPath"/"$fileName"converted."$fileExtension"

### <font size="4">» </font><font size="5">Extract + Upload Frames from Video</font>

In [ ]:
import os, re, time, pathlib
import urllib.request
from IPython.display import clear_output

Auto_UP_Gdrive = False  
AUTO_MOVE_PATH = "/content" 
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    runSh,
    findProcess,
    loadingAn,
    updateCheck,
    ngrok
)

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!mkdir -p "/content/frames"

for i in range(10):
    clear_output()
    loadingAn()
    print("Uploading Frames...")

%cd "/content/frames"
!ffmpeg -hide_banner -ss 00:56.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame1.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame1.png" https://catbox.moe/user/api.php -o frame1.txt
f1 = open('frame1.txt', 'r')
%cd "/content"
file_content1 = f1.read()

%cd "/content/frames"
!ffmpeg -hide_banner -ss 02:20.0 -i "$inputFile" -vframes 1 -q:v 1 -y "/content/frames/frame2.png"
!curl --silent -F "reqtype=fileupload" -F "fileToUpload=@frame2.png" https://catbox.moe/user/api.php -o frame2.txt
%cd "/content/frames"
f2 = open('frame2.txt', 'r')
%cd "/content"
file_content2 = f2.read()

clear_output()
print ("Screenshot URLs:")
print ("1. " + file_content1)
print ("2. " + file_content2)


### <font size="4">» </font><font size="5">MediaInfo</font>

In [ ]:
path = "" #@param {type:"string"}
save_txt = False #@param {type:"boolean"}
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output, drive

def mediainfo():
  display(HTML("<br>"))
#   print(path.split("/")[::-1][0])
  display(HTML("<br>"))
#   media = !mediainfo "$path"
#   media = "\n".join(media).replace(os.path.dirname(path)+"/", "")
  get_ipython().system_raw("""mediainfo --LogFile="/root/.nfo" "$path" """)
  with open('/root/.nfo', 'r') as file:
    media = file.read()
    media = media.replace(os.path.dirname(path)+"/", "")
  print(media)
  get_ipython().system_raw("rm -f '/root/.nfo'")
  
  if save_txt:
    txt = path.rpartition('.')[0] + ".txt"
    if os.path.exists(txt):
      get_ipython().system_raw("rm -f '$txt'")
    !curl -s https://pastebin.com/raw/TApKLQfM -o "$txt"
    with open(txt, 'a+') as file:
      file.write("\n\n")
      file.write(media)

while not os.path.exists("/content/drive"):
  try:
    drive.mount("/content/drive")
    clear_output(wait=True)
  except:
    clear_output()
    
if not os.path.exists("/usr/bin/mediainfo"):
  get_ipython().system_raw("apt-get install mediainfo")
  
mediainfo()

## <img src='https://raw.githubusercontent.com/sudo-ken/Torrents-to-Google-Drive/master/title_qbittorrent.png' height="30" alt="qBittorrent"/>

###**<font color="red">Update</font> - replaced ngrok with localtunnel**
* Install localtunnel **first** and then install qbittorrent, then it'll give you a URL for the WEB-UI.

In [ ]:
#@markdown <h3>INSTALL LOCALTUNNEL</h3>
!npm install -g localtunnel


In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 INSTALL QBITTORRENT</h3>
import os, time, urllib.request, json; from IPython.display import clear_output


if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    get_ipython().system_raw("apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
    get_ipython().system_raw("apt install qbittorrent-nox")
    get_ipython().system_raw("mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/7TEALGNz -o /root/.config/qBittorrent/qBittorrent.conf")
    print("qBittorrent successfully installed.")
    clear_output(wait=True)
else:
    print("qBittorrent already installed. Skipping...")
    clear_output(wait=True)
    !pkill qbittorrent-nox
    
!qbittorrent-nox -d --webui-port=5454
print("qBittorrent started ")
clear_output(wait=True)

if os.path.isfile("/usr/bin/rclone") == False:
    get_ipython().system_raw('curl https://rclone.org/install.sh | sudo bash &')
    print("Rclone successfully installed")
    clear_output(wait=True)
else:
    print("Rclone already installed. Skipping...")
    clear_output(wait=True)

!lt --port 5454

## <img src='https://raw.githubusercontent.com/sudo-ken/Torrents-to-Google-Drive/master/title_qbittorrent.png' height="30" alt="qBittorrent"/>

###**<font color="red">Update</font> - (NGROK Version)**
* WEB-UI.

**Username:- admin**

**Password:- adminadmin**



In [ ]:
#@title ⬅️ Run This Cell for qBittorrent v4.1.7 Web UI
Ngrok_Token = "" #@param {type:"string"}
import os, time, urllib.request, json; from IPython.display import clear_output
if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    get_ipython().system_raw("apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
    get_ipython().system_raw("apt install qbittorrent-nox")
    get_ipython().system_raw("mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/XbthBkJU -o /root/.config/qBittorrent/qBittorrent.conf")
    print("qBittorrent successfully installed.")
    clear_output(wait=True)
else:
    print("qBittorrent already installed. Skipping...")
    clear_output(wait=True)
    !pkill qbittorrent-nox
    
!qbittorrent-nox -d --webui-port=4444
print("qBittorrent started ")
clear_output(wait=True)

if os.path.isfile("/usr/local/bin/ngrok") == False:
  !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -qq -n ngrok-stable-linux-amd64.zip
  !mv ngrok /usr/local/bin/ngrok
  !rm -f /content/ngrok-stable-linux-amd64.zip
  print("ngrok successfully installed.")
  clear_output(wait=True)
else:
  print("ngrok already installed. Skipping... ")
  clear_output(wait=True)
  !pkill ngrok
  
if os.path.isfile("/usr/bin/rclone") == False:
    get_ipython().system_raw('curl https://rclone.org/install.sh | sudo bash &')
    print("Rclone successfully installed")
    clear_output(wait=True)
else:
    print("Rclone already installed. Skipping...")
    clear_output(wait=True)

get_ipython().system_raw('/usr/local/bin/ngrok authtoken $Ngrok_Token && /usr/local/bin/ngrok http 4444 &')
print("Hold on...")
time.sleep(10)
clear_output(wait=True)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  host = data['tunnels'][0]['public_url'][8:]
  print(f'Open this link in new tab http://{host}')

## <img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="30" alt="Peerflix-server" /> __PEERFLIX-SERVER__

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://camo.githubusercontent.com/95b0ba2a237ee846b3bb8f45954124eb9cba4ebc/68747470733a2f2f63646e2e6a7364656c6976722e6e65742f67682f617361706163682f70656572666c69782d736572766572406d61737465722f6170702f696d616765732f6c6f676f2e737667' height="50" alt="netdata"/></center>
#@markdown <center><h3>PEERFLIX-SERVER<br />Streaming torrent client for node.js with web ui.</h3></center><br>

import os
import time
import pathlib
import urllib.request
from json import dump
from IPython.display import clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    updateCheck,
    runSh,
    loadingAn,
    findProcess,
    PortForward_wrapper
)

#####################################
if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()
pathlib.Path('peerflix').mkdir(mode=0o777, exist_ok=True)
loadingAn()

#CONFIG
if findProcess("node", "peerflix-server"):
  !pkill -f "node /tools/node/bin/peerflix-server"
configPath = f"{HOME}/.config/peerflix-server/config.json"
data = {}  
trackers = urllib.request.urlopen(
          "https://trackerslist.com/best.txt"
      ).read().decode().split("\n\n")
data['connections'] = 500
data['path'] = "peerflix" 
data['dht'] = True 
data['tracker'] = True
data['trackers'] = trackers
try:
  os.mkdir(f'{HOME}/.config/peerflix-server/')
except:
  pass
with open(configPath, "w+") as configFile:
  dump(data, configFile)

if not os.path.isfile("/tools/node/bin/peerflix-server"):
  runSh('npm install -g peerflix-server')
  
#Opening Peerflix-server in background
if not findProcess("node", "peerflix-server"):
  PORT = 4445
  cmdC = f'PORT={PORT} peerflix-server &'
  for run in range(10):    
    runSh(cmdC, shell=True)
    time.sleep(3)
    try:
      urllib.request.urlopen(f"http://localhost:{PORT}")
      break
    except:
      print("Error: Peerflix-server not starting. Retrying ...")

# START_SERVER
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['simple-torrent', 4444, 'http'], 
    ['peerflix-server', 4445, 'http']], 'us', 
    [f"{HOME}/.ngrok2/sTpS.yml", 4040]
).start('peerflix-server', 'r')


In [ ]:

#@markdown <br><center><img src='https://i.imgur.com/YWSbyoZ.png' height="50" alt="netdata"/></center>
#@markdown <br><center><h1>CLEAN DOWNLODAS FOLDER</center>
#@markdown <br><center>PEERFLIX-SERVER don't delete removed torrent task file. So you do this manually after remove torrent task.
!touch /content/downloads/errorRemove
!cd downloads/ && sudo rm -r *
print("All Deleted on /content/peerflix")

## <img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="30" alt="Deluge" /> __Deluge__

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/c/c5/Deluge_icon.png' height="50" alt="netdata"/></center>
#@markdown <center><h3>Deluge<br />Deluge is a BitTorrent client written in Python.</h3></center><br>
#@markdown <center><h5>Auto upload to gdrive after task completed. if gdrive is mounted<br>Default output path /content/downloads/</h5></center><br>
import os, urllib.request, pathlib
from IPython.display import clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    updateCheck,
    runSh,
    findProcess,
    loadingAn,
    PortForward_wrapper
)
if updateCheck("Checking updates ...", Version):  # VERSION CHECKING ...
    !kill -9 -1 &
clear_output()

loadingAn()

pathlib.Path('downloads').mkdir(exist_ok=True)
pathlib.Path(f"{HOME}/.config/deluge/").mkdir(parents=True, exist_ok=True)

if not (findProcess("/usr/bin/python", "deluged") or findProcess("/usr/bin/python", "deluge-web")):
  runSh('sudo apt install -y deluged deluge-console deluge-webui')
  runSh(
        f"wget -qq https://raw.githubusercontent.com/helloW3c/Google-Colab-CloudTorrent/master/res/deluge/core.conf \
            -O {HOME}/.config/deluge/core.conf"
    )
  runSh(
        f"wget -qq https://raw.githubusercontent.com/helloW3c/Google-Colab-CloudTorrent/master/res/deluge/web.conf \
            -O {HOME}/.config/deluge/web.conf"
    )
  runSh('deluged &> /dev/null &', shell=True)
  runSh('deluge-web --fork', shell=True)
  runSh("""sed -i 's/if s.hexdigest() == config\["pwd_sha1"\]:/if True:/' /usr/lib/python2.7/dist-packages/deluge/ui/web/auth.py""")
  runSh("sed -i 's/onShow:function(){this.passwordField.focus(.*)}/onShow:function(){this.onLogin();}/' /usr/lib/python2.7/dist-packages/deluge/ui/web/js/deluge-all.js")

# START_SERVER
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['deluge', 8112, 'http'],
                            ['utorrent', 5454, 'http']], 'eu', 
    [f"{HOME}/.ngrok2/delugeUtorrent.yml", 4042]
).start('deluge')


##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> __Extracte/Archive Zip/Tar/Rar__

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "7Z"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = "\"/content/drive/MyDrive/"  # @param {type:"string"}
extractPath = "/content/drive/MyDrive"  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = ""  # @param {type:"string"}
PATH_TO_FILE = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 4#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=2%>Use this to path separate between to this character `|`. Ex `path/to /1 | path/to/2` </br>#Split only in zip mode.(Only for noobs)
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else "" 
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


In [ ]:
#@markdown <h3>INSTALL LOCALTUNNEL</h3>
!npm install -g localtunnel


In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 INSTALL QBITTORRENT</h3>
import os, time, urllib.request, json; from IPython.display import clear_output


if os.path.isfile("/usr/bin/qbittorrent-nox") == False:
    get_ipython().system_raw("apt update -qq -y && yes "" | add-apt-repository ppa:qbittorrent-team/qbittorrent-stable")
    get_ipython().system_raw("apt install qbittorrent-nox")
    get_ipython().system_raw("mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/7TEALGNz -o /root/.config/qBittorrent/qBittorrent.conf")
    print("qBittorrent successfully installed.")
    clear_output(wait=True)
else:
    print("qBittorrent already installed. Skipping...")
    clear_output(wait=True)
    !pkill qbittorrent-nox
    
!qbittorrent-nox -d --webui-port=5454
print("qBittorrent started ")
clear_output(wait=True)

if os.path.isfile("/usr/bin/rclone") == False:
    get_ipython().system_raw('curl https://rclone.org/install.sh | sudo bash &')
    print("Rclone successfully installed")
    clear_output(wait=True)
else:
    print("Rclone already installed. Skipping...")
    clear_output(wait=True)

!lt --port 5454

## <img src='https://github.com/D3VL/L3MON/raw/master/server/assets/webpublic/logo.png' height="30" alt="L3MON" /><font size=1px>___L3MON DEMO___</font>

In [ ]:
#@markdown <br><center><img src='https://github.com/D3VL/L3MON/raw/master/server/assets/webpublic/logo.png' height="30" alt="L3MON"/></center>
#@markdown <center><h3>L3MON<br /> Remote Android Managment Suite</h3></center><br>
import os, pathlib, zipfile, re, json, hashlib, time
import urllib.request
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

#####################################
USE_FREE_TOKEN = False  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AU" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
RUN_Dir = "/content/tools/L3MON"  # @param {type:"string"}
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    textAn
)

loadingAn()

if not RUN_Dir:
  RUN_Dir = "tools/L3MON"
os.makedirs(RUN_Dir, exist_ok=True)


def md5(txt):
  return hashlib.md5(txt.encode('utf-8')).hexdigest()

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of L3MON
if not os.path.exists(f"{RUN_Dir}/index.js"):
  BASE_URL = r"https://github.com/D3VL/L3MON"
  LATEST_TAG = latestTag(BASE_URL)
  urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
              f'L3MON-v{LATEST_TAG}.zip'
  urllib.request.urlretrieve(urlF, f'{RUN_Dir}/newL3MON.zip')
  with zipfile.ZipFile(f'{RUN_Dir}/newL3MON.zip', 'r') as zip_ref:
      zip_ref.extractall(RUN_Dir)
  try:
    pathlib.Path(f'{RUN_Dir}/newL3MON.zip').unlink()
  except FileNotFoundError:
    pass

  # Installing L3MON
  runSh('npm install pm2 -g')

  # Uninstalling latest java
  runSh('apt purge openjdk-\* icedtea-\* icedtea6-\*')
  # Install version 8
  runSh('apt install openjdk-8-jre')

  runSh('npm install', cd=RUN_Dir)
  runSh('pm2 start index.js', cd=RUN_Dir)
  time.sleep(2)
  runSh('pm2 stop index.js', cd=RUN_Dir)
  with open(f'{RUN_Dir}/maindb.json', 'r+') as f:
      data = json.load(f)
      if USERNAME:
        data['admin']['username'] = USERNAME
      if PASSWORD:
        data['admin']['password'] = md5(PASSWORD)
      else:
        data['admin']['password'] = md5('admin')
      f.seek(0)        
      json.dump(data, f, indent=4)
      f.truncate()

if not findProcess('node' f'{RUN_Dir}/index.js'):
  runSh('pm2 restart all', shell=True, cd=RUN_Dir)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['L3MON', 22533, 'http'], 
                            ['L3MON_Client', 22222, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/L3MON.yml", 4051]
)
data = Server.start('L3MON')
data2 = Server.start('L3MON_Client', displayB=False, v=False)
data2 = urlparse(data2['url'])
Host, port = data2.hostname, '80'

display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 20px;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Listening address<p><thead>        <tr>        <th>Host</th>            <th>Port</th> </tr>    </thead>    <tbody>        <tr><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/L3MON_DEMO.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))




## <img src='https://raw.githubusercontent.com/UndeadSec/SocialFishMobile/master/content/logo.png' height="30" alt="SocialFish" /> SocialFish

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/UndeadSec/SocialFishMobile/master/content/logo.png' height="30" alt="SocialFish"/></center>
#@markdown <center><h3>SocialFish<br />Educational Phishing Tool & Information Collector</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output


#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    textAn
)

loadingAn()

os.makedirs('tools/', exist_ok=True)

# Downloading SocialFish
if not os.path.exists("tools/SocialFish/SocialFish.py"):
  clear_output()
  textAn("[Installing] Wait for almost 1 minutes...")
  # runSh('apt install -y texlive-latex-extra')
  runSh('git clone https://github.com/biplobsd/SocialFish.git', cd='tools/')
  runSh('pip install -r requirements.txt', cd='tools/SocialFish')
  os.chmod("tools/SocialFish/SocialFish.py", 0o755)

if not findProcess("python3", "./SocialFish.py"):
  runCmd = "./SocialFish.py " \
    f"{USERNAME if USERNAME else 'admin'} " \
    f"{PASSWORD if PASSWORD else 'admin'} &"
  runSh(runCmd, shell=True, cd='tools/SocialFish/')
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['SocialFish', 5000, 'http']],
    REGION.lower(), 
    [f"{HOME}/.ngrok2/SocialFish.yml", 4063]
).start('SocialFish', displayB=False)

displayUrl(Server, pNamU='SocialFish : ', 
           EcUrl="/neptune")


##<img src='https://handbrake.fr/img/logo.png' height="30" alt="netdata"> __HandBrake__

In [ ]:
#@title <<----<strong>Install HandBrake and rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
HANDBRAKE = True #@param {type:"boolean"}
RCLONE = False #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/content/temp/HandbrakeTemp", exist_ok = True)
makedirs("/root/.config/rclone", exist_ok = True) 
if HANDBRAKE==True:
  !wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip 
  !rm -f ffmpeg-4.2.1-linux-64.zip
  !add-apt-repository ppa:stebbins/handbrake-releases -y 
  !apt-get install -y handbrake-cli 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <center> HandBrake Configuration </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "BATCH" #@param ["SINGLE", "BATCH"]


#@markdown ---
SOURCE = "/content/drives/SKGHD/Anime Downloads/test" #@param {type:"string"}
DESTINATION = "/content/drives/SKGHD/HandBrake" #@param {type:"string"}
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "480p" #@param ["480p", "720p", "1080p"]
Encoder = "x264" #@param ["x264", "x265"]
Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 30 #@param {type:"slider", min:10, max:30, step:1}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
Email_Alert = False #@param {type:"boolean"}
MESSAGE = "HandBrakeCLI_Colab has finished working" #@param ["HandBrake has finished working"] {allow-input: true}
SENDER_ID ="" #@param {type:"string"}
SENDER_PASS="" #@param {type:"string"}
RECEIVER_ID="" #@param {type:"string"}
#@markdown >#####  Using an app specific password is recommended: https://myaccount.google.com/apppasswords

########################################################
import smtplib
import os


formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]        
        if tName.endswith('ts'):
          tName = '[HANDY] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        else:
          tName = '[HANDY] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION=="1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags
  
      
  if os.path.isfile(hTemp):
    print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):    
      print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE + currentFile      
          print(f'\n\n**************** Current File to process: {currentFile}')
          runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()
        

else:
    if SOURCE.endswith(formats):    
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file??")

########### Sending Notification ###############
if Email_Alert == True: 
  from email.mime.multipart import MIMEMultipart 
  from email.mime.text import MIMEText 
  msg = MIMEMultipart()
  msg['From'] = SENDER_ID
  msg['To'] = RECEIVER_ID
  msg['Subject'] = "HandBrakeCLI_Colab has finished executing!" 
  
  msg.attach(MIMEText(MESSAGE))
  s = smtplib.SMTP('smtp.gmail.com', 587) 
  s.starttls() 
  s.login(SENDER_ID, SENDER_PASS)
  s.sendmail(SENDER_ID, RECEIVER_ID, msg.as_string())
  print("Email Alert Sent!")  
  s.quit()


##  **YouTube.com**

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ◄ *Execute YouTube Downloader*
# @markdown 📝 Note: if you want to change an archive file just run this cell again.
Archive = False #@param {type:"boolean"}
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

## <img src='https://nicolargo.github.io/glances/public/images/glances.png' height="30" alt="Glances"/> ___Glances___

In [ ]:
#@markdown <br><center><img src='https://nicolargo.github.io/glances/public/images/glances.png' height="30" alt="Glances"/></center>
#@markdown <center><h3>Glances<br />Glances is a cross-platform system monitoring tool written in Python.</h3></center><br>
import os
import urllib.request
from IPython.display import clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "JP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl
)

loadingAn()

if not os.path.exists("/usr/local/bin/glances"):
  os.system(
      "pip3 install https://github.com/nicolargo/glances/archive/master.zip")
  os.system('pip3 install Bottle')
  os.system(
      "pip3 install 'glances[gpu,ip]'")

if not findProcess("glances", "--webserver"):
  Popen(
      'glances --webserver --port 61208 --time 0 --enable-process-extended \
      --byte --diskio-show-ramfs --fs-free-space \
      --disable-check-update'.split()
      )

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['glances', '61208', 'http']], 
    REGION.lower(), 
    [f"{HOME}/.ngrok2/glances.yml", 31499]
).start('glances', displayB=False)
displayUrl(server, EcUrl="/0", btc='g')


## <img src='http://remote.vuze.com/img/remote_logo_med.png' height="30" alt="vuze webui"/> 

In [ ]:
# ============================= FORM ============================= #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}

# ================================================================ #

import os, psutil, time, urllib.request, pathlib
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/helloW3c/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)

def latestTag():
  import re
  from urllib.request import urlopen
  htmlF = urlopen("http://dev.vuze.com/").read().decode('UTF-8')
  return re.findall(r'\sVuze_(\d{4})\sRelease\s', htmlF)[0]


loadingAn()
if not os.path.exists('tools/vuze/Vuze.jar'):
  os.makedirs('downloads', exist_ok=True)
  os.makedirs('tools/vuze/', exist_ok=True)
  runSh('wget -r --level=1 -np -nH -R index.html -nd -k http://svn.vuze.com/public/client/trunk/uis/lib/',
        cd='tools/vuze/')
  rv = latestTag()
  dlink = f"https://netcologne.dl.sourceforge.net/project/azureus/vuze/Vuze_{rv}/Vuze_{rv}.jar"
  urllib.request.urlretrieve(dlink, 'tools/vuze/Vuze.jar') 

  # All command found in set command ex: java -jar Vuze.jar --ui=console -c set
  runScript = """plugin install xmwebui
pair enable
set "Plugin.xmwebui.Port" 9595 int
set "Plugin.xmwebui.Password Enable" true boolean
set "Plugin.xmwebui.Pairing Enable" false boolean
set "Plugin.xmwebui.User" "root" string
set "Plugin.xmwebui.Password" "yesme" password
set "Completed Files Directory" "/content/downloads/" string
set "General_sDefaultSave_Directory" "/content/downloads/" string
set "General_sDefaultTorrent_Directory" "/content/downloads/" string
"""
  with open('tools/vuze/Rscript.sh', 'w') as w:
    w.write(runScript)

if not findProcess('java', '-jar Vuze.jar'):
  runSh('java -jar Vuze.jar --ui=console -e Rscript.sh &', 
        cd='tools/vuze/',
        shell=True)
  time.sleep(7)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['vuze', 9595, 'http'], 
                            ['transmission', 9091, 'http']], 'sa', 
    [f"{HOME}/.ngrok2/transmissionVuze.yml", 4058]
).start('vuze', displayB=False)
displayUrl(server, pNamU='vuze : ', 
           ExUrl=fr"http://root:yesme@{server['url'][7:]}",
           btc='b')

## <img src='https://avatars1.githubusercontent.com/u/3368377?s=400&v=4' height="30" alt="logo"/><img src='https://raw.githubusercontent.com/linuxserver/docker-templates/master/linuxserver.io/img/nzbget-banner.png' height="52" alt="nzbget"/>

In [ ]:
#@markdown <h3>← Click Here to Use NZBget (<i>ngrok</i>)</h3>
#@markdown <center>Creat an account and Get Free Token  <a href="https://dashboard.ngrok.com/auth">Here</a>  
Ngrok_Token = "" #@param {type:"string"}

import os, psutil, uuid, time, urllib.request, json; from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import HTML, clear_output
from google.colab import output

class MakeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    template = """<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

if os.path.isfile("/usr/local/bin/ngrok") == False:
  !wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -qq -n ngrok-stable-linux-amd64.zip
  !mv ngrok /usr/local/bin/ngrok
  !rm -f /content/ngrok-stable-linux-amd64.zip
  print("ngrok successfully installed.")
  clear_output(wait=True)
else:
  print("ngrok already installed. Skipping... ")
  clear_output(wait=True)
  !pkill ngrok

def RandomGenerator():
  return time.strftime("%S") + str(time.time()).split(".")[-1]

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

try:
  try:
    Random_NumberNZ
  except NameError:
    Random_NumberNZ = RandomGenerator()
    display(MakeLabel('Installing in Progress', 'warning'))
  if os.path.isfile("/content/nzbget/nzbget") == False:
    !mkdir /content/nzbget
    get_ipython().system_raw("rm -rf /content/sample_data/ && wget https://nzbget.net/download/nzbget-latest-bin-linux.run && sh nzbget-latest-bin-linux.run")
  if CheckProcess("nzbget", "") != True:
    get_ipython().system_raw("/content/nzbget/nzbget -s &")
except:
  clear_output(wait=True)

get_ipython().system_raw('/usr/local/bin/ngrok authtoken $Ngrok_Token && /usr/local/bin/ngrok http 6789 &')
time.sleep(10)
clear_output(wait=True)
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  host = data['tunnels'][0]['public_url'][8:]
  print(f'Open this link in new tab http://{host}')

## <img src='https://www.v2ray.com/resources/v2ray_1024.png' height="30" alt="V2Ray" /> __V2Ray__

In [ ]:
# ============================= FORM ============================= #
#@markdown <br><center><img src='https://www.v2ray.com/resources/v2ray_1024.png' height="30" alt="V2Ray"/></center>
#@markdown <center><h3>V2Ray<br />An free and open-source proxy server</h3></center><br>
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
ID=""  # @param {type:"string"}
DEFAULT_SET = True  # @param {type:"boolean"}
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# ================================================================ #
 
import json
import base64
import uuid
import os
import urllib.request
import re
from IPython.display import HTML, clear_output
from subprocess import Popen
import pandas as pd
import requests
 
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")
 
from ocr import PortForward_wrapper, findPackageR, textAn
 
def show_qr(text, v=True):
  try:
    from MyQR import myqr
  except:
    Popen("pip install myqr".split()).wait()
  Popen(["myqr",text]+"--version 5 --level H -n qrcode.png -d /tmp".split()).wait()
  if v:
    from PIL import Image
    im = Image.open("/tmp/qrcode.png")
    os.remove("/tmp/qrcode.png")
    im.show()
    import matplotlib.pyplot as plt
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.axis('off')
    plt.show()
  else:
    with open("/tmp/qrcode.png", "rb") as imageFile:
      imgb64 = base64.b64encode(imageFile.read())
    os.remove("/tmp/qrcode.png")
    return imgb64

def v2ray(id=None,port=9999):
  
  found = findPackageR('v2fly/v2ray-core', 'v2ray-linux-64.zip', all_=True)
  downUrl = found['assets']['browser_download_url']
  print(f"Installing v2ray {found['tag_name']} ...")
  if not os.path.exists("/v2raybin"):
    os.system(f'mkdir -m 777 /v2raybin  &&  cd /v2raybin  && curl -L -H "Cache-Control: no-cache" -o v2ray.zip {downUrl}  && unzip v2ray.zip  && chmod +x /v2raybin/v2ray  && rm -rf v2ray.zip  && chgrp -R 0 /v2raybin  && chmod -R g+rwX /v2raybin')
  CONFIG_JSON1="{\"log\":{\"access\":\"\",\"error\":\"\",\"loglevel\":\"warning\"},\"inbound\":{\"protocol\":\"vmess\",\"port\":"
  CONFIG_JSON2=",\"settings\":{\"clients\":[{\"id\":\""
  CONFIG_JSON3="\",\"alterId\":64}]},\"streamSettings\":{\"network\":\"ws\"}},\"inboundDetour\":[],\"outbound\":{\"protocol\":\"freedom\",\"settings\":{}}}"
  with open("config.json","w") as f:
    f.write(CONFIG_JSON1+str(port)+CONFIG_JSON2+id+CONFIG_JSON3)
  return Popen("/v2raybin/v2ray")
 
port=9910
 
if not ID:
  ID=str(uuid.uuid4())
print("Setting up v2ray server ... ")
v2ray(ID,port)
print("Setting up tunnel ... ")
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['v2ray', port, 'http']],
    REGION.lower(), [f"{HOME}/.ngrok2/V2ray.yml", 8097]
).start('v2ray', displayB=False)
 
d=json.loads('{"add":"{0}","aid":"64","host":"","id":"{1}","net":"ws","path":"","port":"80","ps":"1","tls":"","type":"none","v":"2"}')
d["add"]=re.search("(?<=//).*?(/|$)",Server["url"]).group()
d["id"]=ID
clear_output()
# print(d)
config="vmess://"+base64.b64encode(json.dumps(d).encode()).decode("utf-8")
# print(config)
imgb64 = show_qr(config, v=False).decode()
df_marks = pd.Series(d).to_frame("V2Ray Config")
html = df_marks.to_html(classes="zui-table blueBG")
display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button style="width: 60%" onclick="copy_text_fun()"><img style="width: 40%;" src="data:image/png;base64, """+imgb64+""""/><div style="text-align: -webkit-center;">"""+html+"""</div><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://bit.ly/34E09QG">NB. How to setup this's config. [Click ME]</a></button><center>"""))
display(HTML("""<script type="text/javascript">function copy_text_fun() {var copyText = document.getElementById("copy_txt"); var input = document.createElement("textarea");input.value = copyText.textContent;document.body.appendChild(input);input.select();document.execCommand("Copy");input.remove();}</script><p hidden id="copy_txt">"""+config+"""</p>"""))

## <img src='https://www.booleanworld.com/wp-content/webp-express/webp-images/doc-root/wp-content/uploads/2018/06/wget-cover-300x256@2x.png.webp' height="30" alt="qBitTorrent"/>   ___Wget___<h6>Direct Download / Remote Upload</h6>


In [ ]:
#@title <h3><b>←</b> Run to Download a DDL File
#@markdown <h5><i>Example URL:</i> <a src="">https://files.catbox.moe/archive.zip</a>
direct_URL = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}

!wget "$Direct_URL" "$output_dir"

##  <img src='https://www.deepnude.com/logo_head.png' height="30" alt="RcloneLab"/> ___Deepnude___ <h5>(Need to switch to GPU runtime)</h5>

In [ ]:
#@title Clone repo and install requirements - Need to restart runtime
%cd /content
!git clone https://github.com/XPilot/dreamtime-private.git
%cd '/content/dreamtime-private/src/cli'
f=open('run.py','r').read().replace('create_watermark(nude)','nude');open('run.py','w').write(f)
!pip install -r /content/dreamtime-private/src/cli/requirements-ubuntu.txt

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+https://github.com/jeroenmeulenaar/python3-mega.git')
os.system('pip install mediafire')
os.system('pip install googleDriveFileDownloader')


clear_output()
print("Install Finished")






### I got link here: 
https://gitgud.io/dreamnet/dreamtime#download-mirrors

Also you can find torrent, and manualy upload to Gdrive


In [ ]:
#@title Download checkpoints (pytorch models  *.lib x3)
%cd "/content/dreamtime-private/src/cli"
check_path = '/content/dreamtime-private/src/cli/checkpoints.zip'

Gdrive_url = "https://drive.google.com/uc?id=1p1pbiBkNLyStDqU2VBOMf4kTKJht9dtP&export=download" #@param {type:"string"}
MegaNZ_url = "https://mega.nz/#!3ih2jIgB!EcfD4_K_blnjfNfVmqhV4drrsN_xh4gaMsZzSZIw17s" #@param {type:"string"}
Direct_Link = "" #@param {type:"string"}

Download_from = "Gdrive" #@param ["MegaNZ", "DirectLink","Gdrive"]

from pathlib import Path
if Path(check_path).exists():
  q_delete = input('Already downloaded. Delete zip file? (y)/n')
  if q_delete.lower() == 'y': 
    !rm $check_path

if not Path(check_path).exists():
  print('Downloading started')
  if Download_from == 'MegaNZ':
    from mega import Mega
    m = Mega.from_ephemeral()
    m.download_from_url(MegaNZ_url)

if not Path(check_path).exists():
   if Download_from == 'Gdrive':
    from googleDriveFileDownloader import googleDriveFileDownloader
    a = googleDriveFileDownloader()
    a.downloadFile(Gdrive_url)                            
                                
else:
     import urllib
     urllib.request.urlretrieve ( Direct_Link, "checkpoints.zip" )
     print('Finished')
#пиздец я заебался эту хуйню писать. Надеюсь заработает

def unzip(zip_path, dest_path):
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd
  print("Unziped!")

%cd '/content/dreamtime-private/src/cli'
check_dir_path = '/content/dreamtime-private/src/cli/checkpoints'

from pathlib import Path
if Path(check_dir_path).exists():
  !mkdir $check_dir_path

unzip('checkpoints.zip', 'checkpoints')

### Photo edit

In [ ]:
#@title Upload Photo
%cd /content/dreamtime-private/src/cli

from pathlib import Path
if not Path('photos').exists():
  !mkdir photos

if not Path('temp_photos').exists():
  !mkdir temp_photos

%cd /content/dreamtime-private/src/cli/temp_photos

print('Choose 1 png file')
target = 'test'
from google.colab import files
uploaded = files.upload()

for name, bytes_ in uploaded.items():
  if name.endswith('.png') or name.endswith('.jpg'):
    target=name
    break
  else:
    print('Only image')
 
if target == 'test':
  print('Timeout finished. Pls restart and be faster')
else:
  %matplotlib inline
  from IPython.display import display
  from PIL import Image, ImageDraw
  from ipywidgets import widgets, interact
  import matplotlib.pyplot as plt
  import numpy as np

  image = Image.open(target)
  im_w, im_h = image.size
  im_w+=int(im_w/2)
  im_h+=int(im_h/2)
  size = (im_w, im_h)

  if size == (512, 512):
    print('All right')
    image.save('../photos/'+target[:-3]+'png')
  else:
    print('Pls resize image')
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGBA', size, (255, 255, 255, 0))
    background.paste(
        image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
    )
    image = background

    def f(BoxSize, x, y, save=False):
      ed_image = image.copy()
      draw = ImageDraw.Draw(ed_image)
      draw.rectangle([(x, y), 
                      (x+BoxSize, y+BoxSize)], 
                     outline='red', 
                     width=4)
      if save:
        print('SAVED!')
        crop_im = image.copy()
        crop_im = crop_im.crop((x,y,x+BoxSize,y+BoxSize))
        crop_im = crop_im.resize([512,512], Image.ANTIALIAS)
        crop_im.save('../photos/'+target[:-3]+'png')
      display_im = ed_image.copy()
      display_im.thumbnail((700,700), Image.ANTIALIAS)
      display(display_im)

    def on_butt_clicked(event):
      f(w_box.value,w_x.value,w_y.value,save=True)

    w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
    w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
    w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
    butt = widgets.Button(description='Save')
    butt.on_click(on_butt_clicked)

    display(butt)
    interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@title Upload gif

%cd /content/dreamtime-private/src/cli

from pathlib import Path
if not Path('gifs').exists():
  !mkdir gifs

%cd gifs
from google.colab import files
files.upload()
  

### SEND NUDES!

In [ ]:
#@title Image magic
Pubic_Hair = True #@param {type:"boolean"}
%cd "/content/dreamtime-private/src/cli"


path='results'
import os
if not os.path.exists(path):
    os.makedirs(path)

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

mypath = '/content/dreamtime-private/src/cli/photos'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose file:',
  )
  butt = widgets.Button(description='Start')
  def on_butt_clicked(b):
      print(filename_ch.value)
      %cd '/content/dreamtime-private/src/cli'
      if Pubic_Hair == True:
        !python main.py -i "photos/$filename_ch.value" -o "results/$filename_ch.value" --enablepubes --gpu 0
      else:
        !python main.py -i "photos/$filename_ch.value" -o "results/$filename_ch.value" --gpu 0
      %cd '/content/dreamtime-private/src/cli/results'
      from IPython.display import display, Image
      display(Image(filename_ch.value))
    


  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload photo at first')





  

In [ ]:
#@title Gif magic
Pubic_Hair = True #@param {type:"boolean"}
%cd "/content/dreamtime-private/src/cli"


path='gifs'
import os
if not os.path.exists(path):
    os.makedirs(path)

path='gif_results'
if not os.path.exists(path):
    os.makedirs(path)

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

mypath = '/content/dreamtime-private/src/cli/gifs'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose gif:',
  )
  butt = widgets.Button(description='Start')
  def on_butt_clicked(b):
      print(filename_ch.value)
      %cd '/content/dreamtime-private/src/cli'
      if Pubic_Hair == True:
       !python main.py -i "gifs/$filename_ch.value" -o "gif_results/$filename_ch.value" --enablepubes --gpu 0 --gif
      else:
       !python main.py -i "gifs/$filename_ch.value" -o "gif_results/$filename_ch.value" --gpu 0 --gif
      %cd '/content/dreamtime-private/src/cli/gif_results'
      from IPython.display import display, Image
      display(Image(filename_ch.value))

  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload gif at first')



### Tools

In [ ]:
#@title List files
!ls /content/dreamtime-private/src/cli/temp_photos


In [ ]:
#@title Crop
%cd /content/dreamtime-private/src/cli/temp_photos
target = '' #@param {type:"string"}
%matplotlib inline
from IPython.display import display
from PIL import Image, ImageDraw
from ipywidgets import widgets, interact
import matplotlib.pyplot as plt
import numpy as np

image = Image.open(target)
im_w, im_h = image.size
im_w+=int(im_w/2)
im_h+=int(im_h/2)
size = (im_w, im_h)

if size == (512, 512):
  print('All right')
  image.save('../photos/'+target[:-3]+'png')
else:
  print('Pls resize image')
  image.thumbnail(size, Image.ANTIALIAS)
  background = Image.new('RGBA', size, (255, 255, 255, 0))
  background.paste(
      image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
  )
  image = background

  def f(BoxSize, x, y, save=False):
    ed_image = image.copy()
    draw = ImageDraw.Draw(ed_image)
    draw.rectangle([(x, y), 
                    (x+BoxSize, y+BoxSize)], 
                   outline='red', 
                   width=4)
    if save:
      print('SAVED!')
      crop_im = image.copy()
      crop_im = crop_im.crop((x,y,x+BoxSize,y+BoxSize))
      crop_im = crop_im.resize([512,512], Image.ANTIALIAS)
      crop_im.save('../photos/'+target[:-3]+'png')
    display_im = ed_image.copy()
    display_im.thumbnail((700,700), Image.ANTIALIAS)
    display(display_im)

  def on_butt_clicked(event):
    f(w_box.value,w_x.value,w_y.value,save=True)

  w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
  w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
  w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
  butt = widgets.Button(description='Save')
  butt.on_click(on_butt_clicked)

  display(butt)
  interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@title Rotate image
%cd /content/dreamtime-private/src/cli/temp_photos
target = '' #@param {type:"string"}
ro_deg = -86 #@param {type:"integer"}

from PIL import Image
im_rotate= Image.open(target)
im_rotate=im_rotate.rotate(ro_deg,expand=True)
im_rotate.save(target)

##  <img src='https://www.deepnude.com/logo_head.png' height="30" alt="RcloneLab"/> ___DeepNude with DreamNet improvements.___ <h5>(Need to switch to GPU runtime)</h5>

In [ ]:
#@markdown <h3>← Clone repo and install requirements - Need to restart runtime</h3>

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...Please Wait....", ty='twg')
os.system('cd /content && git clone https://github.com/dreamnettech/dreampower.git && cd dreampower && pip install -r /content/dreampower/requirements.txt')


clear_output()
print("Installed Successfully, Restart Your Notebook")
#====================================
from IPython.display import display
import ipywidgets as widgets
#====================================
button = widgets.Button(description="Restart Notebook")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")
        os._exit(00)

button.on_click(on_button_clicked)


In [ ]:
#@markdown <h3>← Download checkpoints (pytorch models  *.lib x3)</h3>
import os
from IPython.display import clear_output
%cd /content/dreampower
!wget https://github.com/dreamnettech/dreampower-checkpoints/releases/download/0.0.1/v0.0.1.zip
os.rename("v0.0.1.zip","checkpoints.zip")
!unzip "checkpoints.zip"

clear_output()
print("Download Finished")

In [ ]:
#@markdown <h3>← Local Upload Photo edit (Manual Crop option)</h3>
%cd /content/dreampower


from pathlib import Path
if not Path('photos').exists():
  !mkdir photos

if not Path('temp_photos').exists():
  !mkdir temp_photos

%cd /content/dreampower/temp_photos

print('Choose 1 png file')
target = 'test'
from google.colab import files
uploaded = files.upload()

for name, bytes_ in uploaded.items():
  if name.endswith('.png') or name.endswith('.jpg'):
    target=name
    break
  else:
    print('Only image')
 
if target == 'test':
  print('Timeout finished. Pls restart and be faster')
else:
  %matplotlib inline
  from IPython.display import display
  from PIL import Image, ImageDraw
  from ipywidgets import widgets, interact
  import matplotlib.pyplot as plt
  import numpy as np

  image = Image.open(target)
  im_w, im_h = image.size
  im_w+=int(im_w/2)
  im_h+=int(im_h/2)
  size = (im_w, im_h)

  if size == (512,512):
    print('All right')
    image.save('../photos/'+target[:-3]+'png')
  else:
    print('Pls resize image')
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGBA', size, (255, 255, 255, 0))
    background.paste(
        image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
    )
    image = background

    def f(BoxSize, x, y, save=False):
      ed_image = image.copy()
      draw = ImageDraw.Draw(ed_image)
      draw.rectangle([(x, y), 
                      (x+BoxSize, y+BoxSize)], 
                     outline='red', 
                     width=4)
      if save:
        print('SAVED!')
        crop_im = image.copy()
        crop_im = crop_im.crop((x,y,x+BoxSize,y+BoxSize))
        crop_im = crop_im.resize([512,512], Image.ANTIALIAS)
        crop_im.save('../photos/'+target[:-3]+'png')
      display_im = ed_image.copy()
      display_im.thumbnail((700,700), Image.ANTIALIAS)
      display(display_im)

    def on_butt_clicked(event):
      f(w_box.value,w_x.value,w_y.value,save=True)

    w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
    w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
    w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
    butt = widgets.Button(description='Save')
    butt.on_click(on_butt_clicked)

    display(butt)
    interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@markdown <h3>← Local Upload Photo edit (Overlay Option)</h3>
%cd /content/dreampower


from pathlib import Path
if not Path('photos').exists():
  !mkdir photos

if not Path('temp_photos').exists():
  !mkdir temp_photos

%cd /content/dreampower/temp_photos

print('Choose 1 png file')
target = 'test'
from google.colab import files
uploaded = files.upload()

for name, bytes_ in uploaded.items():
  if name.endswith('.png') or name.endswith('.jpg'):
    target=name
    break
  else:
    print('Only image')
 
if target == 'test':
  print('Timeout finished. Pls restart and be faster')
else:
  %matplotlib inline
  from IPython.display import display
  from PIL import Image, ImageDraw
  from ipywidgets import widgets, interact
  import matplotlib.pyplot as plt
  import numpy as np

  image = Image.open(target)
  im_w, im_h = image.size
  im_w+=int(im_w/2)
  im_h+=int(im_h/2)
  size = (im_w, im_h)

  if size == (512,512):
    print('All right')
    image.save('../photos/'+target[:-3]+'png')
  else:
    print('Pls resize image')
    image.save('../photos/'+target[:-3]+'png')
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGBA', size, (255, 255, 255, 0))
    background.paste(
         image)
    image = background
  
    

    def f(BoxSize, x, y, save=False):
      ed_image = image.copy()
      draw = ImageDraw.Draw(ed_image)
      draw.rectangle([(x, y), 
                      (x+BoxSize, y+BoxSize)], 
                     outline='red', 
                     width=4)
      if save:
        print('SAVED!')
        overlay_x = x
        overlay_y = y
        overlay_size = BoxSize
        overlay_dims = f'{overlay_x},{overlay_y}:{overlay_x+overlay_size},{overlay_y+overlay_size}'
        print(overlay_dims)
        %cd /content/dreampower
        f = open("overlay.txt","w") 
        f.write(overlay_dims)
        f.close() 
      

      display_im = ed_image.copy()
      display_im.thumbnail((2000,2000), Image.ANTIALIAS)
      display(display_im)
    

    def on_butt_clicked(event):
      f(w_box.value,w_x.value,w_y.value,save=True)

    w_box = widgets.IntSlider(min=100, max=int(image.size[1]/100*70), value=512)
    w_x = widgets.IntSlider(min=0, max=int(image.size[0]/100*70), value=image.size[0]//5)
    w_y = widgets.IntSlider(min=0, max=int(image.size[1]/100*70), value=image.size[1]//5)
    butt = widgets.Button(description='Save')
    butt.on_click(on_butt_clicked)

    display(butt)
    interact(f,  BoxSize=w_box, x=w_x, y=w_y)

In [ ]:
#@markdown <h3>← Send Nudes

Option = "Auto setting for Url upload" #@param ["Web upload","Auto setting for Url upload","local upload","Auto setting for local upload","Overlay","Auto setting for Overlay option"]

if (Option == "Overlay" or Option == "Auto setting for Overlay option"):
 f = open('overlay.txt', 'r')
 overlay_dims = f.read()
 f.close
 print(overlay_dims)


import os
%cd /content/dreampower
os.makedirs("photos", exist_ok=True)
%cd photos
#@markdown Tip - Url only for <b>Web upload & Auto setting for Url upload</b> option
source_image_url = '' #@param {type:"string"}
if (Option == "Web upload" or Option == "Auto setting for Url upload"):
  !wget -N "$source_image_url"
  
Number_of_cpu_core = 8 #@param {type:"slider", min:2, max:8, step:1}
Pubic_Hair_Size = 1 #@param {type:"slider", min:0, max:1.5, step:0.1}

#@markdown ----
#@markdown Tip - If you select <b>Auto settings</b> option, below setting will not effect on output image
Boob_Size = 1.8 #@param {type:"slider", min:0.3, max:2.0, step:0.1}
Areola_Size = 0.7 #@param {type:"slider", min:0.3, max:2.0, step:0.1}
Nipple_Size = 0.7 #@param {type:"slider", min:0.3, max:2.0, step:0.1}
Vagina_Size = 1.5 #@param {type:"slider", min:0.3, max:1.5, step:0.1}

color_transfer = False #@param {type:"boolean"}
os.environ['n'] = str(Number_of_cpu_core)
cpu_string = "--n-cores $n"


if color_transfer == True:
  color_string = "--color-transfer"
else:
  color_string = ""

%cd /content/dreampower

path='results'
import os
if not os.path.exists(path):
    os.makedirs(path)

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

mypath = '/content/dreampower/photos'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

if len(onlyfiles)>0:
  filename_ch = widgets.Dropdown(
      options=onlyfiles,
      description='Choose file:',
  )
  butt = widgets.Button(description='Start')
  def on_butt_clicked(b):
      print(filename_ch.value)
      %cd '/content/dreampower'
      if Option == "Web upload":
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} --auto-resize-crop --hsize "$Pubic_Hair_Size" --gpu 0   
      elif Option == "Auto setting for Url upload":
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} {cpu_string} --auto-resize-crop --hsize "$Pubic_Hair_Size" --gpu 0   
      elif Option == "Auto setting for Overlay option":
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} --overlay "$overlay_dims" --hsize "$Pubic_Hair_Size" --gpu 0   
      elif Option == "Overlay":
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} --overlay "$overlay_dims" --bsize "$Boob_Size" --asize "$Areola_Size" --nsize "$Nipple_Size" --vsize "$Vagina_Size" --hsize "$Pubic_Hair_Size"   --gpu 0  
      elif Option == "local upload":
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} --bsize "$Boob_Size" --asize "$Areola_Size" --nsize "$Nipple_Size" --vsize "$Vagina_Size" --hsize "$Pubic_Hair_Size"   --gpu 0  
      else:
        !python main.py run -i "photos/$filename_ch.value" -o "results/$filename_ch.value" {color_string} --hsize "$Pubic_Hair_Size"   --gpu 0    
      %cd '/content/dreampower/results'
      from IPython.display import display, Image
      display(Image(filename_ch.value))
    


  butt.on_click(on_butt_clicked)
  # display
  display(widgets.VBox([filename_ch,butt]))
else:
  print('Upload photo at first')


##<img src='https://venturebeat.com/wp-content/uploads/2020/03/CORD-19.png?fit=578%2C259&strip=all' height="30" alt="filebrowser" />         ___**Modeling and Control of COVID-19 Outbreak**___

The following cell simulates a COVID-19 outbreak in campus population of size N starting with one infectious case.  The following parameter values for a standard SEIR model were selected from values given in the recent survey by Boldog, et al. (2020).

| Parameter |  Symbol  | Typical |
| :------- | :-- | :-----------: |
| Reproduction number | $R_0$ | 2.4 |
| Incubation period (days)| $\tau_{incubation}$ | 5.1
| Infectious period (days) | $\tau_{infectious}$ | 3.3
| Campus population | $N$ | 20,000 
| Initial number exposed | $n$ | 10
| Mitigation by social distancing | $u$ | 0.2
| Start of social distancing following exposure (weeks) | $t_{sd}$ | 2

To run the model, adjust the sliders below, then click the run arrow that appears in the left marging. The charts will automatically update when the sliders are readjusted.

In [ ]:
#@title SEIR Model with Social Distancing { run: "auto" }
#@markdown 
#@markdown Reproduction number
R0 = 2.4 #@param {type:"slider", min:0.9, max:5, step:0.1}
#@markdown Disease periods in days
t_incubation = 5.1 #@param {type:"slider", min:1, max:14, step:0.1}
t_infective = 3.3 #@param {type:"slider", min:1, max:14, step:0.1}
#@markdown Population Size
N = 50000 #@param {type:"slider", min:1000, max:350000, step: 1000}
#@markdown Initial number exposed
n = 13 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Start of social distancing following exposure (weeks)
t_social_distancing = 0.3 #@param {type:"slider", min:0, max:30, step:0.1}
#@markdown Effectiveness of social distancing (0 to 100%)
u_social_distancing = 41 #@param {type:"slider", min:0, max:100, step:1}

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns

# initial number of infected and recovered individuals
e_initial = n/N
i_initial = 0.00
r_initial = 0.00
s_initial = 1 - e_initial - i_initial - r_initial

alpha = 1/t_incubation
gamma = 1/t_infective
beta = R0*gamma

def step(t):
    return 1 if t >= 7*t_social_distancing else 0

# SEIR model differential equations.
def deriv(x, t, u, alpha, beta, gamma):
    s, e, i, r = x
    dsdt = -(1-u*step(t)/100)*beta * s * i
    dedt =  (1-u*step(t)/100)*beta * s * i - alpha * e
    didt = alpha * e - gamma * i
    drdt =  gamma * i
    return [dsdt, dedt, didt, drdt]

t = np.linspace(0, 210, 210)
x_initial = s_initial, e_initial, i_initial, r_initial
s, e, i, r = odeint(deriv, x_initial, t, args=(u_social_distancing, alpha, beta, gamma)).T
s0, e0, i0, r0 = odeint(deriv, x_initial, t, args=(0, alpha, beta, gamma)).T

# plot the data
fig = plt.figure(figsize=(12, 10))
ax = [fig.add_subplot(311, axisbelow=True), 
      fig.add_subplot(312)]

pal = sns.color_palette()

ax[0].stackplot(t/7, N*s, N*e, N*i, N*r, colors=pal, alpha=0.6)
ax[0].set_title('Susceptible and Recovered Populations with {0:3.0f}% Effective Social Distancing'.format(u_social_distancing))
ax[0].set_xlabel('Weeks following Initial Campus Exposure')
ax[0].set_xlim(0, t[-1]/7)
ax[0].set_ylim(0, N)
ax[0].legend([
    'Susceptible', 
    'Exposed/no symptoms', 
    'Infectious/symptomatic',
    'Recovered'], 
    loc='best')
ax[0].plot(np.array([t_social_distancing, t_social_distancing]), ax[0].get_ylim(), 'r', lw=3)
ax[0].plot(np.array([0, t[-1]])/7, [N/R0, N/R0], lw=3, label='herd immunity')
ax[0].annotate("Start of social distancing",
    (t_social_distancing, 0), (t_social_distancing + 1.5, N/10),
    arrowprops=dict(arrowstyle='->'))
ax[0].annotate("Herd Immunity without social distancing",
    (t[-1]/7, N/R0), (t[-1]/7 - 8, N/R0 - N/5),
    arrowprops=dict(arrowstyle='->'))

ax[1].stackplot(t/7, N*i0,N*e0, colors=pal[2:0:-1], alpha=0.5)
ax[1].stackplot(t/7, N*i, N*e, colors=pal[2:0:-1], alpha=0.5)
ax[1].set_title('Infected Population with no Social Distancing and with {0:3.0f}% Effective Social Distancing'.format(u_social_distancing))
ax[1].set_xlim(0, t[-1]/7)
ax[1].set_ylim(0, max(0.3*N, 1.05*max(N*(e + i))))
ax[1].set_xlabel('Weeks following Initial Campus Exposure')
ax[1].legend([
    'Infective/Symptomatic', 
    'Exposed/Not Sympotomatic'],
    loc='upper right')
ax[1].plot(np.array([t_social_distancing, t_social_distancing]), ax[0].get_ylim(), 'r', lw=3)

y0 = N*(e0 + i0)
k0 = np.argmax(y0)
ax[1].annotate("No social distancing", (t[k0]/7, y0[k0] + 100))

y = N*(e + i)
k = np.argmax(y)
ax[1].annotate("With {0:3.0f}% effective social distancing ".format(u_social_distancing), (t[k]/7, y[k] + 100))

for a in ax:
    a.xaxis.set_major_locator(plt.MultipleLocator(5))
    a.xaxis.set_minor_locator(plt.MultipleLocator(1))
    a.xaxis.set_major_formatter(plt.FormatStrFormatter('%d'))
    a.grid(True)

plt.tight_layout()

##<img src='https://zdnet2.cbsistatic.com/hub/i/r/2015/12/09/5dcc5c80-b9a8-4bea-be99-1d08cbaa6b59/resize/270xauto/ab75ebe72e39435654d713524e970785/screen-shot-2015-12-09-at-09-49-26.png' height="30" alt="filebrowser" /><h5>(Need to switch to GPU runtime)</h5>

In [ ]:
#@markdown <h3>⬅️ Click Here to install hashcat</h3>
!apt install cmake build-essential -y && apt install checkinstall git -y && git clone https://github.com/hashcat/hashcat.git && cd hashcat && git submodule update --init && make && make install 
from IPython.display import HTML, clear_output
clear_output()
print("Server Started Successfully")

In [ ]:
#@markdown <h3>⬅️ Click Here to check Benchmark</h3>(Need to switch to GPU runtime)
!hashcat -b

In [ ]:
!hashcat -h

##<img src='http://pluspng.com/img-png/nvidia-png-nvidia-png-transparent-image-800.png' height="30" alt="filebrowser" /> StyleGAN 2 <h5>(Need to switch to GPU runtime)</h5>

In [ ]:
#@markdown <h3>⬅️ Click Here to Install Require Modules</h3>
%cd /content
!pip install typeguard;
!pip install psutil
!pip install humanize
!pip install tqdm
!rm -rf stylegan2 && git clone https://github.com/NVlabs/stylegan2.git;
%cd /content/stylegan2

print("Installing")

from IPython.display import Image, clear_output
from google.colab import files
import sys
import pickle
import numpy as np
import PIL
import psutil
import humanize
import os
import time
from tqdm import tqdm

from scipy import ndimage

%tensorflow_version 1.x
sys.path.append('/content/stylegan2/dnnlib')
import dnnlib
import dnnlib.tflib as tflib
dnnlib.tflib.init_tf()

entity_to_url = {
  'faces': 'https://drive.google.com/uc?id=1erg93hWnekh57m3cwsAnqJYfYVceVVSe',
  'celebs': 'https://drive.google.com/uc?id=1q8VldTeTbruoh34ih6GftOcybGNA0dcZ',
  'bedrooms': 'https://drive.google.com/uc?id=15EV9JBiQ7ifoi-B-DQAZF4sYPdCAsiCY',
  'cars': 'https://drive.google.com/uc?id=1QzWwIqJITrg5NWG7QyqrArhb_4UhStDy',
  'cats': 'https://drive.google.com/uc?id=1Fz12B8TSPiRtzCqjhFxTH_W-rIZ5rSGr',
  'anime': 'https://drive.google.com/uc?id=1z8N_-xZW9AU45rHYGj1_tDHkIkbnMW-R',
  'chruch': 'https://drive.google.com/uc?id=1-0JMXPdCQLIVxkDE_S9pO8t8mWoEvhHl',
  'horse': 'https://drive.google.com/uc?id=1-1oc3016pUDi2er1zEvjGcFy8FC-QAh3',
  'anime': 'https://drive.google.com/uc?id=1-91fGZSsZJPNlFytg5iHvVLqxKWDLFt8',
  'anime_portrait': 'https://drive.google.com/uc?id=1-Bw24cv9o7qjLtd8yq8bzzz9AjR9QAkL',
  'faces2': 'https://drive.google.com/uc?id=18rJYK9oF6D7C607Be1B_Fu53rjjHUAT1',
  'GOT': 'https://drive.google.com/uc?id=1-0LCuuUxUA0R6gdSd9prn5sP7T01iF0e',
}

model_cache = {}
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=20)

def gen_pil_image(latents, zoom=1, psi=0.7):
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    image = Gs.run(latents, None, randomize_noise=True, output_transform=fmt, truncation_psi=psi)
    if zoom == 1:
        return PIL.Image.fromarray(image[0])
    else:
        print(image[0].shape)
        return PIL.Image.fromarray(ndimage.zoom(image[0],(zoom,zoom,1)))

import google.colab.output
import random
import io
import base64

def gen(l=None, psi=1):
    if l is None:
        l = [random.random()*2-1 for x in range(512)]
    pimg = gen_pil_image(np.array(l).reshape(1,512), psi=psi)
    bio = io.BytesIO()
    pimg.save(bio, "PNG")
    b = bio.getvalue()
    return 'data:image/png;base64,'+str(base64.b64encode(b),encoding='utf-8')

google.colab.output.register_callback('gen', gen)

##
def fetch_model(name):
  if model_cache.get(name):
    return model_cache[name]
  url = entity_to_url[name]
  with dnnlib.util.open_url(url, cache_dir='cache') as f:
    _G, _D, Gs = pickle.load(f)
    model_cache[name] = Gs
  return model_cache[name]

def fetch_file(filename):
  with open(filename,'rb') as f:
    return pickle.load(f)


clear_output()
print("Installed Successfully")

In [ ]:
#@markdown <h3>⬅️ Click Here to Render Model</h3>
#choose model here. default is ffhq
import os
Render_Model = "faces" #@param ["faces","faces2","GOT","celebs","bedrooms","cars","cats","chruch","horse","anime"]


if Render_Model == "faces":
 curr_model = "faces" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "faces2":
 curr_model = "faces2" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "celebs":
 curr_model = "celebs" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "bedrooms":
 curr_model = "bedrooms" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "cars":
 curr_model = "cars" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "cats":
 curr_model = "cats" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "chruch":
 curr_model = "chruch" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "horse":
 curr_model = "horse" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "anime":
 curr_model = "anime" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "anime_portrait":
 curr_model = "anime_portrait" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')

if Render_Model == "GOT":
 curr_model = "GOT" # can be faces, celebs, bedrooms, cars, cats, anime
 Gs = fetch_model(curr_model) # if you uploaded your own file, use fetch_file('path/to/file.pkl')


In [ ]:
#@markdown <h3>⬅️ Click Here to run Model</h3>
from IPython.display import HTML

def get_latent_html(i):
    return """<div class="pure-control-group">
            <label for="l%i">L%03i:</label>
            <input type="number" min="-999.99" max="999.99" step="0.01" id="l%i" value="%.2f" style="background-color: white;">
    </div>""" % (i, i, i, (random.random()*2-1))

def get_latents_html():
    return '\n'.join([get_latent_html(i) for i in range(512)])

input_form = """
<link rel="stylesheet" href="https://necolas.github.io/normalize.css/8.0.1/normalize.css">
<link rel="stylesheet" href="https://unpkg.com/purecss@1.0.1/build/pure-min.css" integrity="sha384-oAOxQR6DkCoMliIh8yFnu25d7Eq/PHS21PClpwjOTeU2jRSq11vu66rf90/cZr47" crossorigin="anonymous">

<div style="background-color:white; border:solid #ccc; width:1200px; padding:20px; color: black;">
<p>You have currently loaded %s model</p>
  <div class="pure-g" style="width:1200px; margin-bottom: 25px;">
    <div class="pure-u-2-3">
      <img id="stylegan" src="" style="height:512px; width:512px;">
    </div>
    <div class="pure-u-1-3">
      <div style="overflow-y:scroll; height:512px; width:300px" class="pure-form pure-form-aligned">
        %s
      </div>
    </div>
  </div>

  <div class="pure-g">
    <div class="pure-u-1-6">
      <button class="pure-button pure-button-primary" onclick="generate();">Generate from latents</button>
    </div>
    <div class="pure-u-1-6 pure-form">
      <div class="pure-control-group">
            <label for="psi">psi:</label>
            <input type="number" min="0" max="999.99" step="0.01" id="psi" value="0.7" style="background-color: white;">
      </div>
    </div>
    <div class="pure-u-1-6">
      <button class="pure-button pure-button-primary" onclick="mutate();">Mutate randomly</button>
    </div>
    <div class="pure-u-1-6 pure-form">
      <div class="pure-control-group">
            <label for="mut_str">Mutation strength:</label>
            <input type="number" min="0" max="999.99" step="0.01" id="mut_str" value="0.2" style="background-color: white;">
      </div>
    </div>
    <div class="pure-u-1-6">
      <button class="pure-button pure-button-primary" onclick="randomize();">Random image</button>
    </div>
    <div class="pure-u-1-6">
      <button class="pure-button pure-button-primary" onclick="nnormalize();">Normalize latents</button>
    </div>
  </div>

  <div class="pure-g">
    <div class="pure-u-1-3">
      <button class="pure-button pure-button-primary" onclick="save();">Save latents</button>
      <button class="pure-button pure-button-primary" onclick="load();">Load latents</button>
    </div>
     <div class="pure-u-2-3 pure-form">
      <div class="pure-control-group">
            <input type="text" id="save-input" style="width:100%%; background-color: white;" placeholder="Saved latents will show up here...">
      </div>
    </div>
  </div>

</div>
""" % (curr_model, get_latents_html())

javascript = """
<img src onerror='generate()'>
<script type="text/Javascript">
    function desanitize(text) {
        return text.slice(1,-1).replace(/\\\\n/g, "\\n").replace(/\\\\'/g, "'");
    };

    function set_img(text) {
        document.getElementById('stylegan').src = text;
    };

    function generate() {
        var kernel = google.colab.kernel;
        var latents = [];
        var psi = parseFloat(document.getElementById('psi').value);
        for (var i=0;i<512;i++) {
            latents[i] = parseFloat(document.getElementById('l'+i).value);
            //console.log(i);
        };
        console.log(latents);
        var resultPromise = kernel.invokeFunction("gen", [latents, psi]);
        resultPromise.then(
            function(value) {
              console.log(value.data);
              set_img(desanitize(value.data["text/plain"]));
              //document.getElementById('spinner').style = "visibility: hidden;";
        });
    };

    function mutate() {
        var kernel = google.colab.kernel;
        var latents = [];
        var psi = parseFloat(document.getElementById('psi').value);
        var mutationStrength = parseFloat(document.getElementById('mut_str').value)
        for (var i=0;i<512;i++) {
            latents[i] = parseFloat(document.getElementById('l'+i).value);
            latents[i] += (Math.random()*2-1) * mutationStrength;
            document.getElementById('l'+i).value = latents[i];
        };
        console.log(latents);
        var resultPromise = kernel.invokeFunction("gen", [latents, psi]);
        resultPromise.then(
            function(value) {
              console.log(value.data);
              set_img(desanitize(value.data["text/plain"]));
              //document.getElementById('spinner').style = "visibility: hidden;";
        });
    };

    function nnormalize() {
        console.log('Any normies?');
        var latents = [];
        for (var i=0;i<512;i++) {
            latents[i] = parseFloat(document.getElementById('l'+i).value);
        };
        var sum = latents.reduce((a, b) => a + b, 0);
        for (var i=0;i<512;i++) {
            latents[i] = latents[i]/sum;
            document.getElementById('l'+i).value = latents[i];
        };
        console.log(latents);
        generate();
    };

    function randomize() {
        var latents = [];
        for (var i=0;i<512;i++) {
            latents[i] = Math.random()*2-1;
            document.getElementById('l'+i).value = latents[i];
        };
        generate();
    };

    function save() {
        var latents = [];
        for (var i=0;i<512;i++) {
            latents[i] = parseFloat(document.getElementById('l'+i).value);
        };
        document.getElementById('save-input').value = JSON.stringify(latents);
    }

    function load() {
        var latents = JSON.parse(document.getElementById('save-input').value);
        for (var i=0;i<512;i++) {
            document.getElementById('l'+i).value = latents[i];
        };
        generate();
    }

</script>
"""

HTML(input_form + javascript)

##<img src='https://upload.wikimedia.org/wikipedia/commons/0/0c/Blender_logo_no_text.svg' height="30" alt="filebrowser" /> Blender <h5>(Need to switch to GPU runtime)</h5>



In [ ]:
#@markdown <h3>⬅️ Click Here to Install Blender</h3>

#@markdown <h3><center><b>Blender Render Script for Colab</b></center></h3>
#@markdown <h5><center>This is a script for rendering in blender using Google Collab</center></h5>

#@markdown <h4>Steps</h4>
#@markdown <h5>- Make sure you have GPU enabled in your kernel (Also ensure that your blend file has GPU enabled)</h5>
#@markdown <h5>- Use wget to download your file into the kernel</h5>
#@markdown <h5>- In the blender command make sure that the correct filename is provided</h5>
#========================================================================================================

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('apt install blender')
os.system('apt install libboost-all-dev')
os.system('apt install libgl1-mesa-dev')

clear_output()
print("Install Finished")

In [ ]:
#@markdown <h3>⬅️ Click Here to Download Blend file to render</h3>
download_file_path = '' #@param {type: "string"}
filename = '' #@param {type: "string"}
filename+= '.blend'
renderer = "CYCLES" #@param ["CYCLES", "BLENDER_RENDER"]
frame = 1  #@param {type: "number"}

!wget -O $filename $download_file_path
##
!ls -la


In [ ]:
#@markdown <h3>⬅️ Click Here to Start Render</h3>
!blender -b $filename -noaudio -o ./test_ -E $renderer -x 1 -f $frame

from IPython.display import Image
Image(filename='./test_0001.png')

##<img src='https://pngimage.net/wp-content/uploads/2018/06/ocr-png-5.png' height="30" alt="filebrowser" /> OCR <h5>



In [ ]:
#@markdown <h3>⬅️ Click Here to Install Require Modules</h3>

#Install these Python Libraries


from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install pytesseract')
os.system('apt install tesseract-ocr-eng')
os.system('apt install tesseract-ocr-mya')
os.system('apt install libtesseract-dev')
os.system('pip3 install opencv-python')
os.system('apt-get install libopencv-dev')

clear_output()
print("Installed Successfully")

In [ ]:
#@markdown <h3>⬅️ Click Here to Start Image OCR</h3>

#Read Image and extract text

from PIL import Image
import PIL.Image
from pytesseract import image_to_string
import pytesseract
ocr_file = '' #@param {type: "string"}
pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'
TESSDATA_PREFIX = '/usr/local/bin/pytesseract'
img = Image.open(ocr_file)


Language = "English" #@param ["Myanmar", "English"]

if Language == 'Myanmar':
 output = pytesseract.image_to_string (img, lang='mya')
else:
 output = pytesseract.image_to_string (img, lang='eng')

print (output)


##<img src='https://scontent.frgn5-1.fna.fbcdn.net/v/t1.0-0/p320x320/10653877_472738456200409_4614784466362332725_n.png?_nc_cat=104&_nc_sid=85a577&efg=eyJpIjoiYiJ9&_nc_ohc=KsxPM460zb8AX_3CmcF&_nc_ht=scontent.frgn5-1.fna&oh=e6e57738399d267807328436c57e848f&oe=5EC2D478' height="30" alt="filebrowser" />  <h5>


<h5>(Need to switch to GPU runtime and Python 3)</h5>


In [ ]:
#@markdown <h3>⬅️ Click Here to Install Colorizer</h3>
#@markdown <center><img src='https://scontent.frgn5-2.fna.fbcdn.net/v/t31.0-8/24059638_680264242361237_3143229723815922153_o.png?_nc_cat=111&_nc_sid=6e5ad9&_nc_ohc=iOB_DdBSuJkAX9cAOCi&_nc_ht=scontent.frgn5-2.fna&oh=c24fc1a2fed7cf41966984d54071ef5c&oe=5EC3E4D1' height="100" alt="filebrowser" /></center> 

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...Wait for almost 60 seconds....", ty='twg')
os.system('cd /content && git clone https://github.com/jantic/DeOldify.git DeOldify')
%cd DeOldify
###
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

os.system('pip install -r colab_requirements.txt')

###
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
###
os.makedirs("models", exist_ok=True)
os.system('wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth')
os.system('wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth')
os.system('wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png')



clear_output()
print("Installed Successfully")


In [ ]:
#@markdown <h3>⬅️ Click Here to Colorize image or video</h3>

import os
from IPython.display import clear_output
#@markdown ⚙ Recommended image sources 
#@markdown * [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)


#@markdown ⚙ Recommended video and gif sources 
#@markdown * [/r/Nickelodeons/](https://www.reddit.com/r/Nickelodeons/)
#@markdown * [r/silentmoviegifs](https://www.reddit.com/r/silentmoviegifs/)
#@markdown * https://twitter.com/silentmoviegifs 

#==================================================

source_url = 'https://i.imgur.com/xgKH8Qb.png' #@param {type:"string"}
render_factor = 8  #@param {type: "slider", min: 7, max: 45}
selection_option = "Image" #@param ["Image", "Video/GIF"]
watermarked = False #@param {type:"boolean"}

if selection_option == "Image":
 colorizer = get_image_colorizer(artistic=True)
 clear_output()
 if source_url is not None and source_url !='':
  image_path = colorizer.plot_transformed_image_from_url(url=source_url, render_factor=render_factor, compare=True, watermarked=watermarked)
  show_image_in_notebook(image_path)
 else:
  print('Provide an image url and try again.')

else:
 colorizer = get_video_colorizer()
 clear_output()
 if source_url is not None and source_url !='':
  video_path = colorizer.colorize_from_url(source_url, 'video.mp4', render_factor, watermarked=watermarked)
  show_video_in_notebook(video_path)
 else:
  print('Provide a video url and try again.')

In [ ]:
#@markdown <h3>⬅️ Click Here to See how well render_factor values perform </h3>

selection_option = "Image" #@param ["Image", "Video/GIF"]

if selection_option == "Image":
 for i in range(10,45,2):
    colorizer.plot_transformed_image('test_images/image.png', render_factor=i, display_render_factor=True, figsize=(8,8))

else:
 for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

##<img src='https://media.istockphoto.com/vectors/identification-face-color-line-icon-eye-id-verifying-recognition-vector-id1172721433' height="30" alt="filebrowser" /> ___*Deepface*___  <h5>
<h5>(Need to switch to GPU runtime )</h5>




### Welcome to DFL-Colab!

This is an adapted version of the DFL for Google Colab.


# Overview
*   Extractor works in full functionality.
*   Training can work without preview.
*   Merger works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Import/export and another manipulations with workspace you can do in "Manage workspace" block
*   Google Colab machine active for 12 hours. DFL-Colab makes a backup of your workspace in training mode.
*   Google does not like long-term heavy calculations. Therefore, for training more than two sessions in a row, use two Google accounts. It is recommended to split your training over 2 accounts, but you can use one Google Drive account to store your workspace.



### Install or update DeepFaceLab

* Install or update DeepFAceLab directly from Github
* Requirements install is automatically
* Automatically sets timer to prevent random disconnects

In [ ]:
#@title Install or update DeepFaceLab from Github

Mode = "install" #@param ["install", "update"]

from pathlib import Path
if (Mode == "install"):
  !git clone https://github.com/iperov/DeepFaceLab.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/DeepFaceLab
  !git pull

!pip install -r /content/DeepFaceLab/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0

if not Path("/content/pretrain").exists():
  print("Downloading CelebA faceset ... ")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/pretrain-CelebA/pretrain_CelebA.zip' -O /content/pretrain_CelebA.zip
  !mkdir /content/pretrain
  !unzip -q /content/pretrain_CelebA.zip -d /content/pretrain/
  !rm /content/pretrain_CelebA.zip

if not Path("/content/pretrain_Q96").exists():
  print("Downloading Q96 pretrained model ...")
  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
  !mkdir /content/pretrain_Q96
  !unzip -q /content/pretrain_Q96.zip -d /content/pretrain_Q96/
  !rm /content/pretrain_Q96.zip

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

### Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [ ]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")



In [ ]:
#@title Export to Drive { form-width: "30%" }
Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "merged", "merged_mask", "models", "result video", "result_mask video"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

def zip_and_copy(path, mode):
  zip_cmd="-r -q "+Archive_name+" "
  
  %cd $path
  zip_cmd+=mode
  !zip $zip_cmd
  copy_cmd = " "+Archive_name+"  /content/drive/My\ Drive/"
  !cp $copy_cmd
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "merged":
  zip_and_copy("/content/workspace/data_dst", "merged")
elif Mode == "merged_mask":
  zip_and_copy("/content/workspace/data_dst", "merged_mask")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
elif Mode == "result video":
  !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/
  
print("Done!")


In [ ]:
#@title Import from URL{ form-width: "30%", display-mode: "form" }
URL = "http://" #@param {type:"string"}
Mode = "unzip to content" #@param ["unzip to content", "unzip to content/workspace", "unzip to content/workspace/data_src", "unzip to content/workspace/data_src/aligned", "unzip to content/workspace/data_dst", "unzip to content/workspace/data_dst/aligned", "unzip to content/workspace/model", "download to content/workspace"]

import urllib
from pathlib import Path

def unzip(zip_path, dest_path):

    
  unzip_cmd = " unzip -q " + zip_path + " -d "+dest_path
  !$unzip_cmd  
  rm_cmd = "rm "+dest_path + url_path.name
  !$rm_cmd
  print("Unziped!")
  

if Mode == "unzip to content":
  dest_path = "/content/"
elif Mode == "unzip to content/workspace":
  dest_path = "/content/workspace/"
elif Mode == "unzip to content/workspace/data_src":
  dest_path = "/content/workspace/data_src/"
elif Mode == "unzip to content/workspace/data_src/aligned":
  dest_path = "/content/workspace/data_src/aligned/"
elif Mode == "unzip to content/workspace/data_dst":
  dest_path = "/content/workspace/data_dst/"
elif Mode == "unzip to content/workspace/data_dst/aligned":
  dest_path = "/content/workspace/data_dst/aligned/"
elif Mode == "unzip to content/workspace/model":
  dest_path = "/content/workspace/model/"
elif Mode == "download to content/workspace":
  dest_path = "/content/workspace/"

if not Path("/content/workspace").exists():
  cmd = "mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"
  !$cmd

url_path = Path(URL)
urllib.request.urlretrieve ( URL, dest_path + url_path.name )

if (url_path.suffix == ".zip") and (Mode!="download to content/workspace"):
  unzip(dest_path + url_path.name, dest_path)

  
print("Done!")

In [ ]:
#@title Export to URL
URL = "http://" #@param {type:"string"}
Mode = "upload workspace" #@param ["upload workspace", "upload data_src", "upload data_dst", "upload data_src aligned", "upload data_dst aligned", "upload merged", "upload model", "upload result video"]

cmd_zip = "zip -r -q "

def run_cmd(zip_path, curl_url):
  cmd_zip = "zip -r -q "+zip_path
  cmd_curl = "curl --silent -F "+curl_url+" -D out.txt > /dev/null"
  !$cmd_zip
  !$cmd_curl


if Mode == "upload workspace":
  %cd "/content"
  run_cmd("workspace.zip workspace/","'data=@/content/workspace.zip' "+URL)
elif Mode == "upload data_src":
  %cd "/content/workspace"
  run_cmd("data_src.zip data_src/", "'data=@/content/workspace/data_src.zip' "+URL)
elif Mode == "upload data_dst":
  %cd "/content/workspace"
  run_cmd("data_dst.zip data_dst/", "'data=@/content/workspace/data_dst.zip' "+URL)
elif Mode == "upload data_src aligned":
  %cd "/content/workspace"
  run_cmd("data_src_aligned.zip data_src/aligned", "'data=@/content/workspace/data_src_aligned.zip' "+URL )
elif Mode == "upload data_dst aligned":
  %cd "/content/workspace"
  run_cmd("data_dst_aligned.zip data_dst/aligned/", "'data=@/content/workspace/data_dst_aligned.zip' "+URL)
elif Mode == "upload merged":
  %cd "/content/workspace/data_dst"
  run_cmd("merged.zip merged/","'data=@/content/workspace/data_dst/merged.zip' "+URL )
elif Mode == "upload model":
  %cd "/content/workspace"
  run_cmd("model.zip model/", "'data=@/content/workspace/model.zip' "+URL)
elif Mode == "upload result video":
  %cd "/content/workspace"
  run_cmd("result.zip result.mp4", "'data=@/content/workspace/result.zip' "+URL)
  
  
!rm *.zip

%cd "/content"
print("Done!")

In [ ]:
#@title Delete and recreate
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged; rm -r /content/workspace/data_dst/merged_mask"
  
!$cmd
print("Done!")

### Extract, sorting and faceset tools
* Extract frames for SRC or DST video.
* Denoise SRC or DST video. "Factor" param set intesity of denoising
* Detect and align faces. If you need, you can get frames with debug landmarks.
* Export workspace to Google Drive after extract and sort it manually (In "Manage Workspace" block)
* You can enhance your facesets with DFL FacesetEnhancer.
* Apply or remove trained XSeg model to the extracted faces


In [ ]:
#@title Extract frames
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "DeepFaceLab/main.py videoed extract-video"

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --output-dir workspace/data_src/"
  
!python $cmd

In [ ]:
#@title Denoise frames
Data = "data_src" #@param ["data_src", "data_dst"]
Factor = 1 #@param {type:"slider", min:1, max:20, step:1}

cmd = "DeepFaceLab/main.py videoed denoise-image-sequence --input-dir workspace/"+Data+" --factor "+str(Factor)

%cd "/content"
!python $cmd

In [ ]:
#@title Detect faces
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD" #@param ["S3FD", "S3FD (whole face)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "DeepFaceLab/main.py extract --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "whole face" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content"
!python $cmd

In [ ]:
#@title Sort aligned
Data = "data_src" #@param ["data_src", "data_dst"]
sort_type = "hist" #@param ["blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-faster", "absdiff"]

cmd = "DeepFaceLab/main.py sort --input-dir workspace/"+Data+"/aligned --by "+sort_type

%cd "/content"
!python $cmd

In [ ]:
#@title Faceset Enhancer
Data = "data_src" #@param ["data_src", "data_dst"]

data_path = "/content/workspace/"+Data+"/aligned"
cmd = "/content/DeepFaceLab/main.py facesettool enhance --input-dir "+data_path
!python $cmd

In [ ]:
#@title Apply or remove XSeg mask to the faces
Mode = "Remove mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]

main_path = "/content/DeepFaceLab/main.py "
data_path = "/content/workspace/"+Data+"/aligned "
mode_arg = "apply " if Mode == "Apply mask" else "remove "
cmd = main_path+"xseg "+mode_arg+"--input-dir "+data_path
cmd += "--model-dir /content/workspace/model" if mode_arg == "apply " else ""

!python $cmd

### Train model

* Choose your model type, but SAEHD is recommend for everyone
* Set model options on output field
* You can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
* Your workspace will be archived and upload to mounted Drive after 11 hours from start session
* If you select "Backup_every_hour" option, your workspace will be backed up every hour.
* Also, you can export your workspace manually in "Manage workspace" block
* "Silent_Start" option provides to automatically start with best GPU and last used model. 

In [ ]:
#@title Training
Model = "SAEHD" #@param ["SAEHD", "Quick96", "XSeg"]
Backup_every_hour = True #@param {type:"boolean"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((43200-uptime)/3600))+" hours")
  backup_time = str(3600)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/3600))+" hours")
  backup_time = str(round(39600-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "DeepFaceLab/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

### Merge frames

In [ ]:
#@title Merge
Model = "SAEHD" #@param ["SAEHD", "Quick96" ]

cmd = "DeepFaceLab/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Get result video 
Mode = "result video" #@param ["result video", "result_mask video"]
Copy_to_Drive = True #@param {type:"boolean"}


if Mode == "result video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio
  if Copy_to_Drive:
    !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python DeepFaceLab/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4
  if Copy_to_Drive:
    !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/


##<img src='https://raw.githubusercontent.com/idealo/image-super-resolution/master/figures/butterfly.png' height="30" alt="filebrowser" /> ___*Image Super-Resolution (ISR)*___ 
<h5>(Need to switch to GPU runtime )</h5> 

In [ ]:
#@markdown <h3>←  Click Here to Install Image Super-Resolution (ISR)</h3>
#@markdown <center><img src='https://raw.githubusercontent.com/idealo/image-super-resolution/master/figures/butterfly.png' height="100" alt="filebrowser" /></center> 

from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...Please Wait....", ty='twg')
os.system('cd /content && pip -q install ISR gdown')

## Get the pre-trained weights and data

#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/ArtefactCancelling/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/PSNR-driven/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C3-D10-G64-G064-x2/PSNR-driven/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5
#!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rrdn-C4-D3-G32-G032-T10-x4/Perceptual/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5

import os
if not os.path.exists('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf'):
# download pretrained weights
 !gdown https://drive.google.com/uc?id=1Sn9FFKXNWOPXdNzy-zmyVYB4eNRnXTKD
 !gdown https://drive.google.com/uc?id=1_1GGoHF5oq3W_iVVxx1SRDIa_mjwoe26
 !gdown https://drive.google.com/uc?id=1ro0Rk4xAqo-OEDGgLIYz-F0BHvmUUT2E
 !gdown https://drive.google.com/uc?id=1o3l_I60xHkdiWZG7UM0nZBblEontHs2W
 !mkdir weights
 !mv *.hdf5 weights
#=========
clear_output()
print("Sucessfully Installed")



In [ ]:
#@markdown <h3>←  Getty Image</h3>
from IPython.display import clear_output
import os


source_image_url = '' #@param {type:"string"}
!wget -O test.png "$source_image_url"



os.makedirs("data/input/test_images", exist_ok=True)
!mv *.png data/input/test_images


clear_output()
print("Sucessfully Input Image")

import numpy as np
from PIL import Image

img = Image.open('data/input/test_images/test.png')
img

In [ ]:
#@markdown <h3>← Get predictions <i>(Method 1)</i></h3>
### Create the model and run prediction

select_option = "Large RDN noise cancelling, detail enchancing model" #@param ["RRDN GANS mode", "Large RDN model", "Small RDN model", "Large RDN noise cancelling, detail enchancing model"]


from ISR.models import RDN, RRDN
if select_option == 'RRDN GANS model':

 rrdn = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'x':4, 'T': 10})
 rrdn.model.load_weights('weights/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5')

#### Large RDN model
elif select_option == 'Large RDN model':
 rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')

#### Small RDN model
elif select_option == 'Small RDN model':
 rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')

#### Large RDN noise cancelling, detail enchancing model
else:
 rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')

#### Run prediction
sr_img = rdn.predict(np.array(img))
Image.fromarray(sr_img)


In [ ]:
#@markdown <h3>← Get predictions <i>(Method 2)</i></h3>
#@markdown Usecase: upscaling noisy images

#### So let's first  compress the image
img.save('data/input/test_images/compressed.jpeg','JPEG', dpi=[300, 300], quality=50)
compressed_img = Image.open('data/input/test_images/compressed.jpeg')

select_option = "Large RDN noise cancelling, detail enhancing model" #@param ["Large RDN model (PSNR trained)", "Small RDN model  (PSNR trained)", "Large RDN noise cancelling, detail enhancing model"]


compressed_img
# Baseline
# Bicubic scaling
compressed_img.resize(size=(compressed_img.size[0]*2, compressed_img.size[1]*2), resample=Image.BICUBIC)
#### Large RDN model (PSNR trained)
if select_option == 'Large RDN model (PSNR trained)':
 rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')
#### Small RDN model  (PSNR trained)
if select_option == 'Small RDN model  (PSNR trained)':
 rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')
#### Large RDN noise cancelling, detail enhancing model
else:
 rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
 rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')


sr_img = rdn.predict(np.array(compressed_img))
Image.fromarray(sr_img)

## <img src='https://www.wikihow.com/images/thumb/0/04/Miku1and2.png/728px-Miku1and2.png.webp' height="30" alt="filebrowser" /> ___*Waifu2x*___ <p>(Need to switch to GPU runtime )</p>


In [ ]:
#@markdown <h3>←  Click Here to Install Waifu2x</h3>
#@markdown <center><img src='https://raw.githubusercontent.com/tsurumeso/waifu2x-chainer/master/images/summery.png' height="100" alt="filebrowser" /></center> 


from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...Please Wait....", ty='twg')
os.system('cd /content && pip install -q cupy-cuda100 && pip install -q futures && pip install -q chainer && git clone -l -s https://github.com/tsurumeso/waifu2x-chainer.git waifu2x-chainer && cd content/waifu2x-chainer')


clear_output()
print("Sucessfully Installed")






In [ ]:
#@markdown <h3>←  Getty Image</h3>
import os, sys, re
import google.colab.files
%cd /content/waifu2x-chainer
os.makedirs("Input_images", exist_ok=True)
os.makedirs("Output_images", exist_ok=True)
%cd /content/waifu2x-chainer/Input_images
clear_all_files_in_input_folder_after_processing = True #@param {type:"boolean"}
local_upload = False #@param {type:"boolean"}
source_image_url = '' #@param {type:"string"}
if local_upload == True:
 uploaded = google.colab.files.upload()
else:
 !wget "$source_image_url"

#@markdown ---
#@markdown <h5><b>Setting</b></h5>
#Rescale_all_files_in_input_folder = False #@param {type:"boolean"}
#Input_images_file_path = '/content/waifu2x-chainer/Output_images/sad-anime-girl-11579904620pho2n5ujpp.png' #@param {type:"string"}
#Output_image_folder_url = '' #@param {type:"string"}
# - Tip : if you untick rescale all files, you will be ask to choose the file

from IPython.display import display
from ipywidgets import widgets
from os import listdir
from os.path import isfile, join

#@markdown - Tip : arch (VGG7=0,UpConv7=1,ResNet10=2,UpResNet10=3)
arch = "1" #@param ["0","1","2","3"]
quality = 100 #@param {type:"slider", min:0, max:100, step:1}
scale_ratio = 2 #@param {type:"slider", min:0, max:20, step:1}
#batch_size
#block_size
extension  = "png" #@param ["png","webp"]
method = "noise_scale" #@param ["noise","scale","noise_scale"] 
noise_level = 1 #@param {type:"slider", min:0, max:3, step:1}
tta_level = "8" #@param ["2","4","8"]

os.environ['q'] = str(quality)
os.environ['s'] = str(scale_ratio)
os.environ['e'] = extension
os.environ['a'] = arch
os.environ['m'] = method
os.environ['n'] = str(noise_level)
os.environ['T'] = tta_level
#============================================
%cd /content/waifu2x-chainer
#if Rescale_all_files_in_input_folder == True:
!python waifu2x.py -m "$m" -i "Input_images" -o "Output_images" -g 0 -s "$s" -a "$a" -n "$n" -q "$ql" -e "$e" -T "$T"

!zip -r /content/output.zip /content/waifu2x-chainer/Output_images

#else:
  #!python waifu2x.py -m "$m" -i "$Input_images_file_path" -o "Output_images_folder_path" -g 0 -s "$s" -a "$a" -n "$n" -q "$ql" -e "$e" -T "$T"

import glob
if clear_all_files_in_input_folder_after_processing  == True:     
 files = glob.glob('/content/waifu2x-chainer/Input_images/*')
 for f in files:
    os.remove(f)     
    
  #!python main.py run -i "/content/waifu2x-chainer/Input_images/$filename_ch.value" -o "results/$filename_ch.value" --ignore-size --hsize "$Pubic_Hair_Size" --gpu 0 



##<img src='https://i.kym-cdn.com/photos/images/original/001/297/263/909.jpg_large' height="30" alt="filebrowser" /> ___*Image Animation*___ <p>First Order Motion Model for Image Animation (Need to switch to GPU runtime )</p>








In [ ]:
#@markdown <h3>←  Click Here to Install Face Swap</h3>
#@markdown <center><img src='https://i.kym-cdn.com/photos/images/original/001/297/263/909.jpg_large' height="100" alt="filebrowser" /></center> 
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "Google-Colab-CloudTorrent/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...Please Wait....", ty='twg')
os.system('cd /content && git clone https://github.com/AliaksandrSiarohin/first-order-model && cd /content/first-order-model')



!gdown https://drive.google.com/uc?id=1YJ0DDoEy7McI4RDvjPt1JSMsgWJQDUaP
!unzip *.zip


clear_output()
print("Sucessfully Installed")

In [ ]:
#@markdown <h3>←  Load driving video and source image</h3>
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")

#@markdown **Create a model and load checkpoints**
#@markdown - checkpoint example file : vox-cpk.pth.tar
checkpoint_path = '/content/first-order-model/first-order-motion-model/vox-cpk.pth.tar' #@param {type:"string"}
#@markdown - config example file : vox-256.yaml
config_path = '/content/first-order-model/config/vox-256.yaml' #@param {type:"string"}


#@markdown ---
#@markdown **Input your data**
source_image_path = '/content/first-order-model/first-order-motion-model/777.png' #@param {type:"string"}
driving_video_path = '/content/first-order-model/first-order-motion-model/08.mp4' #@param {type:"string"}
driving_video_need_to_crop = False #@param {type:"boolean"}

from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path, checkpoint_path)



if driving_video_need_to_crop == True:
 !ffmpeg -i "$driving_video_path" -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 cropped.mp4
 clear_output()
 source_image = imageio.imread(source_image_path)
 driving_video = imageio.mimread('cropped.mp4')
else:
 source_image = imageio.imread(source_image_path)
 driving_video = imageio.mimread(driving_video_path)

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


#Resize image and video to 256x256

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())



## <img src='https://www.ligh.de/pics/VapourSynth_FilmClip.png' height="30" alt="RcloneLab"/>*VapourSynth*



### Init


In [ ]:
#@title Check GPU
#@markdown Run this to connect to a Colab Instance, and see what GPU Google gave you.

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print(gpu[1])
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

In [ ]:
#@title Setup {display-mode: "form"}
#@markdown Run this to install VapourSynth, VapourSynth plugins and scripts, as well as some example upscaling models.
#NOTE: running this more than once may or may not work. 
#The buggy console output is due to the threaded installing
#Currently TPU support is broken and incomplete, but it isn't particularly useful since it doesn't support opencl anyway 

#Init
import os, sys, shutil, tempfile
import collections
from datetime import datetime, timedelta
import requests
import threading
import ipywidgets as widgets
from IPython import display
import PIL
from google.colab import files
import time
%cd /


#Function defs
#---------------------------------------------------------

#Like shutil.copytree(), but doesn't complain about existing directories
#Note this is fixed in newer version of Python 3
def copytree(src, dst, symlinks=False, ignore=None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytree(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

#Download and extract the .py scripts from the VapourSynth fatpack
def download_fatpack_scripts():
  %cd /
  print("Downloading VS FatPack Scripts...")
  dlurl = r"https://github.com/theChaosCoder/vapoursynth-portable-FATPACK/releases/download/r3/VapourSynth64Portable_2019_11_02.7z"
  with tempfile.TemporaryDirectory() as t:
    dpath = os.path.join(t, "VapourSynth64Portable_2019_11_02.7z")
    os.chdir(t)
    !wget {dlurl}
    %cd /
    !7z x -o{t} {dpath}
    scriptsd = os.path.abspath(os.path.join(t, "VapourSynth64Portable", "Scripts"))
    s = os.path.normpath("VapourSynthImports")
    os.makedirs(s, exist_ok = True)
    copytree(scriptsd, s)
    sys.path.append(s)
  
  #Get some additional scripts.
  !wget -O /VapourSynthImports/muvsfunc_numpy.py https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/muvsfunc_numpy.py
  !wget -O /VapourSynthImports/edi_rpow2.py https://gist.githubusercontent.com/YamashitaRen/020c497524e794779d9c/raw/2a20385e50804f8b24f2a2479e2c0f3c335d4853/edi_rpow2.py
  !wget -O /VapourSynthImports/BMToolkit.py https://raw.githubusercontent.com/IFeelBloated/BlockMatchingToolkit/master/BMToolkit.py
  if accelerator == "CUDA":
    !wget -O /VapourSynthImports/Alpha_CuPy.py https://raw.githubusercontent.com/AlphaAtlas/VapourSynth-Super-Resolution-Helper/master/Scripts/Alpha_CuPy.py
    !wget -O /VapourSynthImports/dpid.cu https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/examples/Dpid_cupy/dpid.cu
    !wget -O /VapourSynthImports/bilateral.cu https://raw.githubusercontent.com/WolframRhodium/muvsfunc/master/Collections/examples/BilateralGPU_cupy/bilateral.cu

  #Get an example model:
  import gdown
  gdown.download(r"https://drive.google.com/uc?id=1KToK9mOz05wgxeMaWj9XFLOE4cnvo40D", "/content/4X_Box.pth", quiet=False)

def getdep1():
  %cd /
  #Install apt-fast, for faster installing
  !/bin/bash -c "$(curl -sL https://git.io/vokNn)"
  #Get some basic dependancies
  !apt-fast install -y -q -q subversion davfs2 p7zip-full p7zip-rar ninja-build 

#Get VapourSynth and ImageMagick built just for a colab environment
def getvs():
  %cd /
  #%cd var/cache/apt/archives
  #Artifacts hosted on bintray. If they fail to install, they can be built from source. 
  !curl -L "https://dl.bintray.com/alphaatlas100/vapoursynth-colab/imagemagick_7.0.9-8-1_amd64.deb" -o /var/cache/apt/archives/imagemagick.deb
  !dpkg -i /var/cache/apt/archives/imagemagick.deb
  !ldconfig /usr/local/lib
  !curl -L "https://dl.bintray.com/alphaatlas100/vapoursynth-colab/vapoursynth_48-1_amd64.deb" -o /var/cache/apt/archives/vapoursynth.deb
  !dpkg -i /var/cache/apt/archives/vapoursynth.deb
  !ldconfig /usr/local/lib
  #%cd /

def getvsplugins():
  %cd /
  #Allow unauthenticated sources
  if not os.path.isfile("/etc/apt/apt.conf.d/99myown"):
    with open("/etc/apt/apt.conf.d/99myown", "w+") as f:
      f.write(r'APT::Get::AllowUnauthenticated "true";')
  sources = "/etc/apt/sources.list"
  #Backup original apt sources file, just in case
  with tempfile.TemporaryDirectory() as t:
    tsources = os.path.join(t, os.path.basename(sources))
    shutil.copy(sources, tsources)
    #Add deb-multimedia repo
    #Because building dozens of VS plugins is not fun, and takes forever
    with open(sources, "a+") as f:
      deb = "deb https://www.deb-multimedia.org sid main non-free\n"
      if not "deb-multimedia" in f.read():
        f.write(deb)

    with open(sources, "a+") as f:
      #Temporarily use Debian unstable for some required dependencies 
      if not "ftp.us.debian.org" in f.read():
        f.write("deb http://ftp.us.debian.org/debian/ sid main\n")
    !add-apt-repository -y ppa:deadsnakes/ppa
    !apt-fast update -oAcquire::AllowInsecureRepositories=true
    !apt-fast install -y --allow-unauthenticated deb-multimedia-keyring
    !apt-fast update  

    #Parse plugins to install
    out = !apt-cache search vapoursynth
    vspackages = ""
    #exclude packages with these strings in the name
    exclude = ["waifu", "wobbly", "editor", "dctfilter", "vapoursynth-dev", "vapoursynth-doc"]
    for line in out:
      p =  line.split(" - ")[0].strip()
      if not any(x in p for x in exclude) and "vapoursynth" in p and p != "vapoursynth":
        vspackages = vspackages + p + " "
    print(vspackages)
    #Install VS plugins and a newer ffmpeg build
    !apt-fast install -y --allow-unauthenticated --no-install-recommends ffmpeg youtube-dl libzimg-dev {vspackages} libfftw3-3 libfftw3-double3 libfftw3-dev libfftw3-bin libfftw3-double3 libfftw3-single3 checkinstall
    #Get a tiny example video
    !youtube-dl -o /content/enhance.webm -f 278 https://www.youtube.com/watch?v=I_8ZH1Ggjk0
    #Restore original sources
    os.remove(sources)
    shutil.copy(tsources, sources)
  #Congrats! Apt may or may not be borked.
  copytree("/usr/lib/x86_64-linux-gnu/vapoursynth", "/usr/local/lib/vapoursynth")
  !ldconfig /usr/local/lib/vapoursynth

#Install vapoursynth python modules
def getpythonstuff():
  %cd /
  !python3.6 -m pip install vapoursynth meson opencv-python

def cudastuff():
    %cd /
    out = !nvcc --version
    cudaver = (str(out).split("Cuda compilation tools, release ")[1].split(", ")[0].replace(".", ""))
    #Note this download sometimes times out
    !python3.6 -m pip install mxnet-cu{cudaver} #cupy-cuda{cudaver}
    !pip install git+https://github.com/AlphaAtlas/VSGAN.git

    #Mxnet stuff
    
    modelurl = "https://github.com/WolframRhodium/Super-Resolution-Zoo/trunk"
    if os.path.isdir("/NeuralNetworks"):
      !svn update --set-depth immediates /NeuralNetworks
      !svn update --set-depth infinity /NeuralNetworks/ARAN
    else:
      !svn checkout --depth immediates {modelurl} /NeuralNetworks

def makesrcd(name):
  %cd /
  srpath = os.path.abspath(os.path.join("/src", name))
  os.makedirs(srpath, exist_ok = False)
  %cd {srpath}

def mesongit(giturl):
  p = os.path.basename(giturl)[:-4]
  makesrcd(p)
  !git clone {giturl}
  %cd {p}
  !meson build
  !ninja -C build
  !ninja -C build install

#Taken from https://stackoverflow.com/a/31614591
#Allows exceptions to be caught from threads
from threading import Thread

class PropagatingThread(Thread):
    def run(self):
        self.exc = None
        try:
            if hasattr(self, '_Thread__target'):
                # Thread uses name mangling prior to Python 3.
                self.ret = self._Thread__target(*self._Thread__args, **self._Thread__kwargs)
            else:
                self.ret = self._target(*self._args, **self._kwargs)
        except BaseException as e:
            self.exc = e

    def join(self):
        super(PropagatingThread, self).join()
        if self.exc:
            raise self.exc
        return self.ret


#Interpolation experiment
#%cd /
#os.makedirs("/videotools")
#%cd /videotools
#!git clone https://github.com/sniklaus/pytorch-sepconv.git
#%cd /

#Function for testing vapoursynth scripts
#Takes the path of the script, and a boolean for generating a test frame.

#-----------------------------------------------------------

#Init functions are threaded for speed
#"PropagatingThread" class is used to return exceptions from threads, otherwise they fail silently

t1 = PropagatingThread(target = getdep1)
t1.start()
print("apt init thread started")

t2 = PropagatingThread(target = download_fatpack_scripts)
t2.start()
print("VS script downloader thread started.")

#Get rid of memory usage log spam from MXnet
os.environ["TCMALLOC_LARGE_ALLOC_REPORT_THRESHOLD"] = "107374182400"

#Check for an accelerator
accelerator = None
gpu = None
if 'COLAB_TPU_ADDR' in os.environ:
  #WIP
  raise Exception("TPUs are (currently) not supported! Please use a GPU or CPU instance.")
else:
  #Check for Nvidia GPU, and identify it 
  out = !command -v nvidia-smi
  if out != []:
    out = !nvidia-smi
    for l in out:
      if "Driver Version" in l:
        accelerator = "CUDA"
        print("Nvidia GPU detected:")
        gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
        gpu = gpu[1]
        #print("Tesla K80 < Tesla T4 < Tesla P100")
        break
if accelerator == None:
  print("Warning: No Accelerator Detected!")

t1.join()
print("Apt init thread done.")

t1 = PropagatingThread(target = getvs)
t1.start()
print("Vapoursynth/Imagemagick downloader thread started.")
t1.join()
print("Vapoursynth/Imagemagick installed")

t3 = PropagatingThread(target = getpythonstuff)
t3.start()
print("Pip thread started")

t1 = PropagatingThread(target = getvsplugins)
t1.start()
print("VS plugin downloader thread started.")

t3.join()
print("pip thread done")

if accelerator == "TPU":
  #WIP!
  pass

elif accelerator == "CUDA":
  t3 = PropagatingThread(target = cudastuff)
  t3.start()
  print("CUDA pip thread started.")
else:
  pass

t2.join()
print("VS script downloader thread done.")

t3.join()
print("CUDA pip thread done.")

t1.join()
print("VS plugin thread done.")



#Build some more plugins(s)
#TODO: Build without changing working directory, or try the multiprocessing module, so building can run asynchronously 
print("Building additional plugins")
mesongit(r"https://github.com/HomeOfVapourSynthEvolution/VapourSynth-DCTFilter.git")
mesongit(r"https://github.com/HomeOfVapourSynthEvolution/VapourSynth-TTempSmooth.git")

googpath = None
%cd /

Clear_Console_Output_When_Done = True #@param {type:"boolean"}
if Clear_Console_Output_When_Done:
  display.clear_output()
#if gpu is not None:
#  print(gpu[1])
#  print("A Tesla T4 or P100 is significantly faster than a K80")
#  print("And the K80 doesn't support hardware encoding.")


In [ ]:
#@title Mount Google Drive
#@markdown Highly recommended!

import os
%cd /

#Check if Google Drive is mounted, and mount if its not.
googpath = os.path.abspath(os.path.join("gdrive", "My Drive"))
if not os.path.isdir(googpath):
  from google.colab import drive
  drive.mount('/gdrive', force_remount=True)

In [ ]:
#@title Mount a Nextcloud Drive

import os
nextcloud = "/nextcloud"
os.makedirs(nextcloud, exist_ok=True)
Nextcloud_URL = "https://us.hostiso.cloud/remote.php/webdav/" #@param {type:"string"}

%cd /
if os.path.isfile("/etc/fstab"):
  os.remove("/etc/fstab")
with open("/etc/fstab" , "a") as f:
  f.write(Nextcloud_URL + " " + nextcloud + " davfs user,rw,auto 0 0")
!mount {nextcloud}

### Processing

In [ ]:
%%writefile /content/autogenerated.vpy

#This is the Vapoursynth Script!
#Running this cell will write the code in this cell to disk, for VSPipe to read.
#Later cells will check to see if it executes.
#Edit it just like a regular python VS script.
#Search for functions and function reference in http://vsdb.top/, or browse the "VapourSynthImports" folder. 

#Import functions
import sys, os, cv2
sys.path.append('/VapourSynthImports')
import vapoursynth as vs
import vsgan as VSGAN
import mvsfunc as mvf
#import muvsfunc as muf
#import fvsfunc as fvf
import havsfunc as haf
import Alpha_CuPy as ape
import muvsfunc_numpy as mufnp
#import BMToolkit as bm
import G41Fun as G41
#import vsutil as util
#import edi_rpow2 as edi
#import kagefunc as kage
#import lostfunc as lost
#import vsTAAmbk as taa
#import xvs as xvs
from vapoursynth import core

#Set RAM cache size, in MB
core.max_cache_size = 10500

#Get Video(s) or Image(s). ffms2 (ffmpeg) or imwri (imagemagick) will read just about anything.
#Lsmash sometimes works if ffms2 failes, d2v reads mpeg2 files
clip = core.ffms2.Source(r"/content/enhance.webm")
#clip = core.lsmas.LWLibavSource("/tmp/%d.png")
#clip = core.imwri.Read("testimage.tiff")

#Store source for previewing
src = clip

#Convert to 16 bit YUV for preprocessing
#clip = core.resize.Spline36(clip, format = vs.YUV444P16)

#Deinterlace
#clip = G41.QTGMC(clip, Preset='Medium')

#Mild deblocking
#clip = fvf.AutoDeblock(clip)

#Convert to floating point RGB
clip = mvf.ToRGB(clip, depth = 32)

#Spatio-temportal GPU denoiser. https://github.com/Khanattila/KNLMeansCL/wiki/Filter-description
clip = core.knlm.KNLMeansCL(clip, a = 8, d = 4, h = 1.4)


preupscale = clip
#Run ESRGAN model. See https://upscale.wiki/wiki/Model_Database
vsgan_device = VSGAN.VSGAN()
vsgan_device.load_model(model=r"/content/4X_Box.pth", scale=4)
clip = vsgan_device.run(clip=clip, chunk = False,  pad = 16)

clip = core.knlm.KNLMeansCL(clip, a = 7, d = 3, h = 1.4)



#Run MXNet model. See the "MXNet" cell.
#Tensorflow models are also supported!
#sr_args = dict(model_filename=r'/NeuralNetworks/ARAN/aran_c0_s1_x4', up_scale=4, device_id=0, block_w=256, block_h=128, is_rgb_model=True, pad=None, crop=None, pre_upscale=False)
#clip = mufnp.super_resolution(clip, **sr_args)

#HQ downscale on the GPU with dpid
#clip = ape.GPU_Downscale(clip, width = 3840, height = 2160)

#Convert back to YUV 444 format/Rec 709 colorspace
clip = core.resize.Spline36(clip, format = vs.YUV444P16, matrix_s = "709")

#Strong temporal denoiser and stabilizer with the LR as a motion reference clip, for stabilizing.
prefilter = core.resize.Spline36(preupscale, format = clip.format, width = clip.width, height = clip.height, matrix_s = "709")
clip = G41.SMDegrain(clip, tr=3, RefineMotion=True, pel = 1, prefilter = prefilter)

#Another CPU denoiser/stabilizer. "very high" is very slow.
#clip = haf.MCTemporalDenoise(clip, settings = "very high", useTTmpSm = True, maxr=4, stabilize = True)

#Stabilized Anti Aliasing, with some GPU acceleration
#clip = taa.TAAmbk(clip, opencl=True, stabilize = 3)

#Example sharpeners that work well on high-res images
#Masks or mvf.limitfilter are good ways to keep artifacts in check
#clip = core.warp.AWarpSharp2(clip)
#clip = G41.NonlinUSM(clip, z=3, sstr=0.28, rad=9, power=1)

#High quality, strong debanding
#clip = fvf.GradFun3(clip, smode = 2)

#Convert back to 8 bit YUV420 for output. 
clip = core.resize.Spline36(clip, format = vs.YUV420P8, matrix_s = "709", dither_type = "error_diffusion")

#Interpolate to double the source framerate
#super = core.mv.Super(inter)
#backward_vectors = core.mv.Analyse(super, isb = True,  overlap=4, search = 3)
#forward_vectors = core.mv.Analyse(super, isb = False, overlap=4, search = 3)
#inter = core.mv.FlowFPS(inter, super, backward_vectors, forward_vectors, num=0, den=0)

#Stack the source on top of the processed clip for comparison
src = core.resize.Point(src, width = clip.width, height = clip.height, format = clip.format)
#clip = core.std.StackVertical([clip, src])
#Alternatively, interleave the source and slow down the framerate for easy comparison.
clip = core.std.Interleave([clip, src])
clip = core.std.AssumeFPS(clip, fpsnum = 2)

#clip = core.std.SelectEvery(clip=clip, cycle=48, offsets=[0,1])

clip.set_output()

In [ ]:
#@title Preview Options
#@markdown Run this cell to check the .vpy script, and set preview options. 
#@markdown * Software encoding is relatively slow on colab's single CPU core, but returns a smaller video.
#@markdown * Hardware encoding doesn't work on older GPUs or a TPU, but can be faster.
#@markdown * Sometimes video previews don't work. Chrome seems more reliable than Firefox, but its video player doesn't support scrubbing. Alternatively, you can download the preview in the "/content" folder with the Colab UI.
#@markdown * HEVC support in browsers is iffy.
#@markdown * PNG previews are more reliable, but less optimal.  
#@markdown * In video previews, you can interleave the source and processed clips and change the framerate for easy comparisons. 
#@markdown ***

#TODO: Make vpy file path editable
vpyscript = "/content/autogenerated.vpy"
#@markdown Use hardware encoding.
Hardware_Encoding = True #@param {type:"boolean"}
#@markdown Encode preview as lossless or high quality lossy video
Lossless = False #@param {type:"boolean"}
#@markdown Use HEVC instead of AVC for preview. Experimental.
HEVC = False #@param {type:"boolean"}
#@markdown Generate a single PNG instead of a video.
Write_PNG = False #@param {type:"boolean"}
#@markdown Don't display any video preview, just write it to /content
Display_Video = False #@param {type:"boolean"}
#@markdown Number of preview frames to generate
preview_frames =  120 #@param {type:"integer"}
#Check script with test frame (for debugging)
Test_Frame = False 

from IPython.display import clear_output
import ipywidgets as widgets
from pprint import pprint



def checkscript(vpyfile, checkoutput):
  
  #Clear the preview cache folder, as the script could have changed
  
  quotepath = r'"' + vpyfile + r'"'
  print("Testing script...")
  if checkoutput:
  #See if the script will really output a frame
    test = !vspipe -y -s 0 -e 0 {quotepath} .
  #Parse the script, and return information about it. 
  rawinfo = !vspipe -i {quotepath} -
  #Store clip properties as a dict
  #I really need to learn regex...
  clipinfo = eval(r"{" + str(rawinfo)[1:].replace(r"\n", r"','").replace(r": ", r"': '")[:-1] + r"}")
  !clear
  if not isinstance(clipinfo, dict):
    print(rawinfo)
    raise Exception("Error parsing VapourSynth script!")
  #print("Script output properties: ")
  #!echo {clipinfo}
  return clipinfo, rawinfo, quotepath

#Make a preview button, and a frame slider
#Note that the slider won't appear with single frame scripts
%cd /
#display.clear_output()
!clear
clipinfo, rawinfo, quotepath = checkscript(vpyscript, Test_Frame)
frameslider = None
drawslider = int(clipinfo["Frames"]) > 1
if drawslider:
  frameslider = widgets.IntSlider(value=0, max=(int(clipinfo["Frames"]) - 1), layout=widgets.Layout(width='100%', height='150%'))
else:
  preview_frames = 1
fv = None


if not(preview_frames > 0 and preview_frames <= int(clipinfo["Frames"])):
  raise Exception("preview_frames must be a valid integer")
if drawslider:
  fv = int(frameslider.value)
else:
  fv = 0

encstr = ""
previewfile = r"/usr/local/share/jupyter/nbextensions/preview.mp4"
if os.path.isfile(previewfile):
  os.remove(previewfile)
ev = min((int(fv + preview_frames - 1), int(clipinfo["Frames"])- 1))
enctup = (Hardware_Encoding, HEVC, Lossless) 
if enctup == (True, True, True):
  encstr = r"-c:v hevc_nvenc -profile main10 -preset lossless -spatial_aq:v 1 -aq-strength 15 "
elif enctup == (True, True, False):
  encstr = r"-c:v hevc_nvenc -pix_fmt yuv420p10le -preset:v medium -profile:v main10 -spatial_aq:v 1 -aq-strength 15 -rc:v constqp -qp:v 9"
elif enctup == (True, False, True):
  encstr = r"-c:v h264_nvenc -preset lossless -profile high444p -spatial-aq 1 -aq-strength 15"
elif enctup == (False, True, True):
  encstr = r"-c:v libx265 -pix_fmt yuv420p10le -preset slow -x265-params lossless=1"
elif enctup == (True, False, False):
  encstr = r"-c:v h264_nvenc -pix_fmt yuv420p -preset:v medium -rc:v constqp -qp:v 10 -spatial-aq 1 -aq-strength 15"
elif enctup == (False, False, True):
  encstr = r"-c:v libx264 -preset veryslow -crf 0"
elif enctup == (False, True, False):
  encstr = r"-c:v libx265 -pix_fmt yuv420p10le -preset slow -crf 9"
elif enctup == (False, False, False):
  encstr = r"-c:v libx264 -pix_fmt yuv420p -preset veryslow -crf 9"
else:
  raise Exception("Invalid parameters!")
clear_output()
print(*rawinfo, sep = ' ')
print("Select the frame(s) you want to preview with the slider and 'preview frames', then run the next cell.")
display.display(frameslider)



In [ ]:
#@title Generate Preview

import os, time


previewdisplay = r"""
<video controls autoplay>
  <source src="/nbextensions/preview.mp4" type='video/mp4;"'>
  Your browser does not support the video tag.
</video>
"""
previewpng = "/content/preview" + str(frameslider.value) + ".png"
if os.path.isfile(previewfile):
  os.remove(previewfile)
if os.path.isfile(previewpng):
  os.remove(previewpng)
frames = str(clipinfo["Frames"])
end = min(frameslider.value + preview_frames - 1, int(clipinfo["Frames"]) - 1)
if Write_PNG:
  !vspipe -y -s {frameslider.value} -e {frameslider.value} /content/autogenerated.vpy - | ffmpeg-y -hide_banner -loglevel warning -i pipe: {previewpng} 
  if os.path.isfile(previewpng):
    import PIL
    display.display(PIL.Image.open(previewpng, mode='r'))
  else:
    raise Exception("Error generating preview!")
else:
  out = !vspipe --progress -y -s {frameslider.value} -e {end} /content/autogenerated.vpy - | ffmpeg -y -hide_banner -progress pipe:1 -loglevel warning -i pipe: {encstr} {previewfile} | grep "fps"
  if os.path.isfile(previewfile):
    if os.path.isfile("/content/preview.mp4"):
      os.remove("/content/preview.mp4")
    !ln {previewfile} "/content/preview.mp4"
    clear_output()
    for temp in out:
      if "Output" in temp:
        print(temp)
    if Display_Video:
      display.display(display.HTML(previewdisplay))
  else:
    raise Exception("Error generating preview!")

### Scratch Space


---



In [ ]:
#Do stuff here

#Example ffmpeg script:

!vspipe -y /content/autogenerated.vpy - | ffmpeg -i pipe: -c:v hevc_nvenc -profile:v main10 -preset lossless -spatial_aq:v 1 -aq-strength 15 "/gdrive/My Drive/upscale.mkv"

#TODO: Figure out why vspipe's progress isn't showing up in colab.

### Extra Functions


In [ ]:
#@title Build ImageMagick and VapourSynth for Colab
#@markdown VapourSynth needs to be built for Python 3.6, and Imagemagick needs to be built for the VapourSynth imwri plugin. The setup script pulls from bintray, but this cell will rebuild and reinstall them if those debs dont work. 
#@markdown The built debs can be found in the "src" folder.

#Get some requirements for building
def getbuildstuff():
  !apt-fast install software-properties-common autoconf automake libtool build-essential cython3 coreutils pkg-config
  !python3.6 -m pip install tesseract cython

#Build imagemagick, for imwri and local image manipulation, and create a deb
def buildmagick():
  makesrcd("imagemagick")
  !wget https://imagemagick.org/download/ImageMagick-7.0.9-8.tar.gz
  !tar xvzf ImageMagick-7.0.9-8.tar.gz
  %cd ImageMagick-7.0.9-8
  !./configure --enable-hdri=yes --with-quantum-depth=32
  !make -j 4 --quiet
  !sudo checkinstall -D --fstrans=no --install=yes --default --pakdir=/src --pkgname=imagemagick --pkgversion="8:7.0.9-8"
  !ldconfig /usr/local/lib

#Build vapoursynth for colab (python 3.6, Broadwell SIMD, etc.), and create a deb
def buildvs():
  makesrcd("vapoursynth")
  !wget https://github.com/vapoursynth/vapoursynth/archive/R48.tar.gz
  !tar -xf R48.tar.gz
  %cd vapoursynth-R48
  !./autogen.sh
  !./configure --enable-imwri
  !make -j 4 --quiet
  !sudo checkinstall -D --fstrans=no --install=yes --default --pakdir=/src --pkgname=vapoursynth --pkgversion=48
  !ldconfig /usr/local/lib
  
getbuildstuff()
buildmagick()
buildvs()

In [ ]:
#@title MXnet
#@markdown This cell will pull pretrained models from https://github.com/WolframRhodium/Super-Resolution-Zoo
#@markdown For usage examples, see [this](https://github.com/WolframRhodium/muvsfunc/blob/master/Collections/examples/super_resolution_mxnet.vpy)
#@markdown and [this](https://github.com/WolframRhodium/Super-Resolution-Zoo/wiki/Explanation-of-configurations-in-info.md)
#Note that there's no release for the mxnet C++ plugin, and I can't get it to build in colab, but the header pulls and installs mxnet and the numpy super resolution function
n = "ESRGAN" #@param {type:"string"}
!svn update --set-depth infinity  NeuralNetworks/{n}

## <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Icons8_flat_upload.svg/800px-Icons8_flat_upload.svg.png' height="30" alt="RcloneLab"/>___Remote Uploading___

In [ ]:
#@markdown <h3>⬅️ Click Here to START SERVER</h3>

import os

%cd /content/drive/My Drive
os.makedirs("DirectDownload", exist_ok=True)
%cd DirectDownload

#####################################

%pip install wget

#####################################
import wget
url = input("Enter Remote Url")
initiateDownload = wget.download(url)
initiateDownload

##

%ls /content/drive/My\ Drive/DirectDownload



## __Remote Uploader__

__Supported filehosts:__
*   MediaFire
*   Zippyshare
*   anonfiles
*   BayFiles



In [ ]:
#@title #__👈 Run this cell to install requirements__##
!echo -n "Installing bayfiles-cli..." && npm i -g bayfiles-cli &> /dev/null && echo " Done!"
!echo -n "Installing anonfile-cli..." && mkdir -p sample_data/anonfile-upload &> /dev/null && git clone https://github.com/awersli99/anonfile-upload.git sample_data/anonfile-upload &> /dev/null && cd sample_data/anonfile-upload &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-ul..." && mkdir -p sample_data/zs-ul &> /dev/null && git clone https://github.com/Sorrow446/ZS-UL.git sample_data/zs-ul &> /dev/null && cd sample_data/zs-ul &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-dl..." && mkdir -p sample_data/zs-dl &> /dev/null && git clone https://github.com/Sorrow446/ZS-DL.git sample_data/zs-dl &> /dev/null && cd sample_data/zs-dl &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing mediafire-ul..." && mkdir -p sample_data/mediafire-ul &> /dev/null && git clone https://github.com/JayBrown/mfcmd.py---MediaFire-uploader.git sample_data/mediafire-ul &> /dev/null && pip install pymediafire &> /dev/null && pip install mediafire &> /dev/null && echo " Done!"

## __Download/Upload__

In [ ]:
#@title #__⬇️ Download from filehosts to cloud storage services__#
#@markdown __Supported filehosts:__
#@markdown *   MediaFire (https://download2280.mediafire.com/xxx...)
#@markdown *   Zippyshare (https://www5.zippyshare.com/v/xxx...)
#@markdown *   anonfiles (https://anonfiles.com/xxx...)
#@markdown *   BayFiles (https://bayfiles.com/xxx...)
URL = "" #@param {type:"string"}
Download_Destination = "" #@param {type:"string"}
url, downdir = URL, Download_Destination

def get_file_info(url):
  splited = url.split("/")
  filename=splited[-1]
  splitedfile = filename.split("_")
  splitedfile2="_".join(splitedfile[0:-1])
  modfilename=splitedfile2+"."+splitedfile[-1]
  return filename, modfilename

if "zippyshare" in url:
  !cd sample_data/zs-dl && python3 zs-dl.py -u $url -o $downdir > /content/sample_data/output.txt
  !tail -n 2 sample_data/output.txt
elif "mediafire" in url:
  !wget $url -P $downdir
elif "anonfiles" or "bayfiles" in url:
  filename, modfilename = get_file_info(url)
  !bayfilesdownload $url -o $downdir
  !echo -n "Please correct the file extention on your own: file_txt -> file.txt"
else:
  print("Error: Unsupported filehost")

In [ ]:
#@title #__⬆️ Upload to filehosts from cloud storage services__#
#@markdown __Successfully uploaded max file size:__
#@markdown *   anonfiles (1950MB)
#@markdown *   Zippyshare (500MB)
#@markdown *   BayFiles (99MB)
FileHost = "anonfiles" #@param ["Zippyshare", "anonfiles", "BayFiles"]
File_Path = "" #@param {type:"string"}
filehost, infile = FileHost, File_Path

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

if filehost == "Zippyshare":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/zs-ul && python3 zs-ul.py -f "$infile" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt
if filehost == "anonfiles":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/anonfile-upload && python3 upload.py "$infile" #> /content/sample_data/output.txt
  #!tail -n 2 sample_data/output.txt
if filehost == "BayFiles":
  filename, pathname = get_file_info(infile)
  print(filename, pathname)
  print("Uploading", filename+"...")
  !cd "$pathname" && bayfilesupload "$filename" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt

In [ ]:
#@title #__⬆️ MediaFire Uploader__#
#@markdown The MediaFire API is pretty buggy, and you might receive an error, because the server hasn't finished calculating the file's checksum yet; in that case, wait a couple of seconds (depending on the filesize) and repeat the command: it will not upload, but instead detect the already uploaded file and return the download URL.
Email_Address = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
File_path = "" #@param {type:"string"}
mfemail, mfpassword = Email_Address, Password

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

filename, pathname = get_file_info(File_path)
print("Uploading", filename+"...")
!cd /content/sample_data/mediafire-ul && python3 mfcmd.py -e $mfemail -p $mfpassword -f $File_path 2>/dev/null
print("Done!")

## <img src='https://raw.githubusercontent.com/butlerx/wetty/main/src/assets/favicon.ico' height="25" alt="Wetty"/> ___Wetty___

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/butlerx/wetty/main/src/assets/favicon.ico' height="50" alt="wetty"/></center>
#@markdown <center><h3>Wetty<br />Terminal in browser over http/https.</h3></center><br>
import os, tarfile
import urllib.request
from IPython.display import clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
)

loadingAn()
os.makedirs('tools/temp', exist_ok=True)

if not os.path.exists("tools/wetty/wetty"):
  ## Build then run
  # os.system("git clone https://github.com/butlerx/wetty.git tools/wetty")
  # Popen(
  #     'npm install'.split(), cwd='tools/wetty').wait()
  # Popen('npm run-script build'.split(), cwd='tools/wetty').wait()
  # Popen('npm i -g'.split(), cwd='tools/wetty').wait()

  ## run now
  wettyBF = 'https://github.com/biplobsd/temp/releases/download/v0.001/wetty.tar.gz'
  fileSN = 'tools/temp/wetty.tar.gz'
  urllib.request.urlretrieve(wettyBF, fileSN)
  with tarfile.open(fileSN, 'r:gz') as t:t.extractall('tools/')
  os.remove(fileSN)

if not findProcess("wetty", "--port"):
  # Popen(
  #     r'wetty --port 4343 --bypasshelmet \
  #     -b "/" -c "/bin/bash"'.split(), 
  #     cwd='/content')
  Popen(
      r'tools/wetty/wetty --port 4343 --bypasshelmet \
      -b "/" -c "/bin/bash"'.split(), 
      cwd='/content')

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['wetty', '4343', 'http']], 
    REGION.lower(), 
    [f"{HOME}/.ngrok2/wetty.yml", 31199]
).start('wetty', displayB=True)

## <img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/appdata/NoMachine/logo_E3oZpVzB_400x400.png' height="30" alt="NoMachine" /> ___NoMachine___

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/appdata/NoMachine/logo_E3oZpVzB_400x400.png' height="50" alt="noMachine"/></center>
#@markdown <center>NoMachine<br><h3>Free Remote Desktop For Everybody</h3></center><br>
import os
import tarfile
import urllib.request
import shutil
import time
from IPython.display import HTML, clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = False
TOKEN = ""  # @param {type:"string"}
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
APT_INSTALL = "apt install -y "
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    textAn,
)

loadingAn()

os.makedirs("tools/nomachine", exist_ok=True)
os.makedirs("/root/.icewm", exist_ok=True)

# password ganarate
try:
  print(f"Found old password! : {password}")
except:
  password = 'nomachine'

clear_output()

start = time.time()
if not os.path.exists("tools/nomachine/NX/bin/nxserver"):
  textAn("Wait for almost 49 seconds. It's doing for NoMachine ready ...")

  runW.system_raw('apt update --quiet --force-yes')

  # Minimal install 
  runW.system_raw(
      'apt install --quiet --force-yes --no-install-recommends \
      icewm x11-xserver-utils firefox xterm pcmanfm')

  # icewm theme
  with open('/root/.icewm/theme', 'w') as w:
    w.write('Theme="NanoBlue/default.theme"')
  
  # with open('/root/.icewm/toolbar', 'w') as w:
  #   w.write('prog "chromium" ! chromium-browser --no-sandbox')

  # nomachine
  staticUrl = "https://github.com/biplobsd/temp/releases/download/6.9.2_1/nomachine_6.9.2_1_x86_64.tar.gz"
  configUrl = "https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/appdata/NoMachine/NXetc.tar.gz"
  
  output_file = 'tools/nomachine/nm.tar.gz'
  config_file = 'tools/nomachine/etc.tar.gz'
  urllib.request.urlretrieve(staticUrl, output_file)
  urllib.request.urlretrieve(configUrl, config_file)
  
  with tarfile.open(output_file, 'r:gz') as t:t.extractall('tools/nomachine')
  runSh('./nxserver --install', cd='tools/nomachine/NX', shell=True)
  runSh('./nxserver --stop', cd='tools/nomachine/NX/bin', shell=True)
  
  shutil.rmtree('tools/nomachine/NX/etc')
  with tarfile.open(config_file, 'r:gz') as t:t.extractall('tools/nomachine/NX')
  os.remove(config_file)
  
  os.remove(output_file)
  runSh('./nxserver --startup', cd='tools/nomachine/NX/bin', shell=True)
  runW.system_raw("echo root:$password | chpasswd")

end = time.time()
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "ngrok"
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN,
                             [['nomachine', 4000, 'tcp']], REGION.lower(), 
               [f"{HOME}/.ngrok2/nomachine.yml", 8459])

data = Server.start('nomachine', displayB=False)
clear_output()
host, port = data['url'][7:].split(':')
user = os.popen('whoami').read()

# Colors
bttxt = 'hsla(10, 50%, 85%, 1)'
btcolor = 'hsla(10, 86%, 56%, 1)'
btshado = 'hsla(10, 40%, 52%, .4)'

display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: """+bttxt+""";   --shadow-color: """+btshado+""";   --btn-color: """+btcolor+""";   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>NoMachine config<p><thead>        <tr><th>Username</th> <th>Password</th><th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr><td>"""+user+"""</td><td>"""+password+"""</td><td>NX</td><td>"""+host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/NoMachine.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))

## __Gclone : A modified version of the rclone__

In [ ]:
#@title  ←  Install Gclone
!bash <(wget -qO- https://git.io/gclone.sh)

In [ ]:
#@title  ←  Upload "rclone.conf"
%cd /root/.config/rclone/
from google.colab import files
uploaded = files.upload()


In [ ]:
#@title  ←  Run Gclone
folde_id1 = "" #@param {type:"string"}
folde_id2 = "" #@param {type:"string"}


!gclone copy gc:{folde_id1} gc:{folde_id2}  --drive-server-side-across-configs

## <img src='https://avatars3.githubusercontent.com/u/10668779?s=200&v=4' height="46" alt="rclone"/><i>__clone__</i>
<h6> ✧  Break The 750GB Limit</h6>




In [ ]:
#@title ← Install Gclone
!bash <(wget -qO- https://git.io/gclone.sh)

In [ ]:
!gclone config

#@markdown <p><b><h3><center> You only need to configure it once, and only one key to restore it after backup</center> </p>
#@markdown <p><b>- Step 1 select n, and then give the remote a new name here with gc </p>
#@markdown <p><b>- Step 2 choice 13, client_id and client_secret are not filled in by default. </p>
#@markdown <p><b>- Step 3 select scope and fill in 1, root_folder_id is not filled in by default. </p>
#@markdown <p><b>- Step 4 open the colab folder on the left and find the right copy path of any json file under Autorclone/accounts. For example = /content/AutoRclone/accounts/02d62cb52b2ac72d2c78c5b1b529b202acaf591a.json </ p>
#@markdown <p><b>- Step 5 service_account_file_path> Fill in /content/AutoRclone/accounts/ </p>
#@markdown <p><b>- Step 6 Edit advanced config? (y/n) Select n .</p>
#@markdown <p><b>- Step 7 Configure this as a team drive? Fill in y.

In [ ]:
#@markdown <h1> <p><b><center><font color="pink">Start copying with Gclone</center></h>
source = "0AL51pdekZf0xUk9PVA" #@param {type:"string"}
destination = "0AFws8i4wNMFVUk9PVA" #@param {type:"string"}
folder = "" #@param {type:"string"}
cmd = f"gclone copy gc:{{{source}}} gc:{{{destination}}}{folder} --drive-server-side-across-configs -v"
!$cmd

#@markdown <p>- folder is the format of the folder to be saved is /Movie/Domestic/Jackie Collection No spaces.</p>
#@markdown <p>- If you are afraid of losing the file, execute it 1 or 2 more times, and automatically skip existing resources. </p>

In [ ]:
#@markdown <h3> <p><center>Gclone deduplication tool</center></h>

folder = "" #@param {type:"string"}

cmd = f"gclone dedupe --dedupe-mode newest gc:{folder}"
!$cmd

In [ ]:
#@markdown <center><h3>One-click backup and restore Gclone configuration file-<font color="red">No configuration is needed after restoring</font></h3></center>

from google.colab import drive
drive.mount('/content/drive')

MODE = "RESTORE" #@param ["BACKUP", "RESTORE"]
#Backup SA!
if MODE == "BACKUP": 
  !rclone copy "/root/.config/rclone/rclone.conf" "/content/drive/My Drive/gclone/"
elif MODE == "RESTORE":
  !rclone copy "/content/drive/My Drive/gclone/" "/root/.config/rclone/"

# <center><h3>Copyright © 2022. All right reserved.</h3><h5><h5>All in one downloader </h5></center></h5>